In [1]:
import os
import json
from openai import OpenAI

In [ ]:
# OpenRouter Configuration
OPENROUTER_API_KEY=''

client = OpenAI(
    base_url='https://openrouter.ai/api/v1',
    api_key=OPENROUTER_API_KEY
)
client

In [37]:
def load_and_minify_taxonomy(file_path):
    """
    Reads the taxonomy file and removes whitespace to save tokens.
    """
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        return json.dumps(data, separators=(',', ':'))
    except FileNotFoundError:
        print(f"Error: Could not find file '{file_path}'")
        return None

In [42]:
def get_defense_data(paragraph, taxonomy_string):
    """
    Extracts 6 columns: Classification (3) + Naming (2) + Piloting (1).
    """

    system_prompt = f"""
    You are a Defense Contract Analyst.
    Your goal is to extract 6 specific data points from the "Input Text".

    --------------------------------------------------------
    PART 1: CLASSIFICATION (Use Taxonomy)
    --------------------------------------------------------
    1. EXTRACT KEYWORDS: Look for specific systems (e.g., "MC-130J", "Project Brown Heron", "DDG-51").
    2. SEARCH TAXONOMY: Check the "definition" fields in the provided JSON for these exact keywords.
    3. MATCH:
       - If keyword found in definition -> Select that Specific Type.
       - Else -> Use best semantic match.

    --------------------------------------------------------
    PART 2: NAMING (Extraction Logic)
    --------------------------------------------------------
    1. System Name (Specific):
       - Extract the EXACT system model, variant, or project name mentioned in the text.
       - Examples: "MC-130J", "Project Brown Heron Data-as-a-Service", "F-35A", "USS Nantucket (LCS 27)".
       - If it's a service/construction contract without a system name, use a descriptive name like "Construction projects".

    2. System Name (General):
       - Identify the broad "Family Name" or "Platform Code".
       - Examples:
         * Text: "MC-130J" -> General: "C-130"
         * Text: "Project Brown Heron" -> General: "Project Brown Heron Data-as-a-Service"
         * Text: "USS Nantucket" -> General: "Freedom-class" (or "LCS")
         * Text: "F-35A" -> General: "F-35 Lightning II"

    --------------------------------------------------------
    PART 3: SYSTEM PILOTING (Strict Rules)
    --------------------------------------------------------
    Select ONE option based on the PRODUCT being contracted, not just the platform.

    - "Crewed": The system itself has a human pilot ON BOARD. (e.g., Fighter Jets, Submarines, Ships).
    - "Uncrewed": The system operates without a human on board. (e.g., UAVs, Satellites, SeaFox Mine Neutralizer).
    - "Not Applicable":
       * Software / Data Services (e.g., Palantir, Cloud).
       * Missiles / Munitions (e.g., AMRAAM, Stinger, 120mm ammo).
       * Infrastructure / Construction.
       * Radar / Sensors (unless it's the platform itself).
       * Components (e.g., Gears, Nacelles) -> Inherit from platform IF it's a major upgrade, otherwise "Not Applicable".

    --------------------------------------------------------
    TAXONOMY JSON:
    {taxonomy_string}
    --------------------------------------------------------

    OUTPUT FORMAT (JSON ONLY):
    {{
      "_reasoning": "Brief explanation of your choices.",
      "Market Segment": "...",
      "System Type (General)": "...",
      "System Type (Specific)": "...",
      "System Name (General)": "...",
      "System Name (Specific)": "...",
      "System Piloting": "..."
    }}
    """

    try:
        completion = client.chat.completions.create(
            model="openai/gpt-4o-mini",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": f"Input Text: {paragraph}"}
            ],
            temperature=0,
            response_format={"type": "json_object"}
        )
        return json.loads(completion.choices[0].message.content)
    except Exception as e:
        return {"error": str(e)}

In [43]:
if __name__ == "__main__":
    # 1. Load the taxonomy (Ensure 'defense_taxonomy.json' is in the same folder)
    taxonomy_file = "/content/taxonomy.json"
    minified_taxonomy = load_and_minify_taxonomy(taxonomy_file)

    if minified_taxonomy:
        # Test Case 1: MC-130J (Crewed Aircraft)
        text_1 = "Palantir USG Inc., Palo Alto, California, has been awarded a $53,899,333 firm-fixed-price modification (P000010) to previously awarded contract FA8806-21-C-0010 for Project Brown Heron Data-as-a-Service. The contract modification provides for a Data-as-a-Service platform that supports three mission areas to automatically ingest data across the Department of the Air Force that informs personnel decisions, planning and operations and space situational awareness and command and control. Work will be performed in Palo Alto, California, and is expected to be completed by March 31, 2023. Fiscal 2022 operations and maintenance funds in the amount of $16,000,000 will be obligated at the time of award. The total cumulative face value of the contract is $175,399,333. Space Systems Command, Los Angeles Air Force Base, El Segundo, California, is the contracting activity."

        # Test Case 2: Palantir (Software - Not Applicable)
        text_2 = "Timken Gears & Services Inc., King of Prussia, Pennsylvania, is awarded a $78,492,998 firm-fixed-price modification to previously awarded contract N0002416C4202 to exercise options for two fiscal 2022 main reduction gear shipsets to support DDG-51 Class Hulls 138 and 139. Work will be performed in Santa Fe Springs, California (77%); Riverside, Missouri (10%); Latrobe, Pennsylvania (3%); Fitchburg, Massachusetts (3%); Erie, Pennsylvania (3%); New Castle, Delaware (2%); Milwaukee, Wisconsin (1%); St. Augustine, Florida (1%), and is expected to be completed by November 2025. Fiscal 2022 shipbuilding and conversion (Navy) funds in the amount of $78,492,998 will be obligated at time of award and will not expire at the end of the current fiscal year. The Naval Sea Systems Command, Washington, D.C., is the contracting activity. "

        # Test Case 3: MQ-9 Reaper (Uncrewed)
        text_3 = "Perikin Enterprises LLC, Albuquerque, New Mexico, has been awarded a $49,338,203 cost-plus-fixed-fee contract for hypersonic test and evaluation advisory and assistance services (A&AS). This contract provides for technical A&AS for hypersonic test and evaluation requirements for the High Speed Systems Test and Hypersonic Central Test and Evaluation Investment Program portfolios. Work will be performed at Arnold Air Force Base, Tennessee, and is expected to be completed by May 31, 2027. This award is the result of a competitive acquisition and two offers were received. Fiscal 2022 research, development, test and evaluation, Air Force funds in the amount of $1,460,021 are being obligated at the time of award. Air Force Test Center, Arnold Air Force Base, Tennessee, is the contracting activity (FA9101-22-C-B004)."

        for i, text in enumerate([text_1, text_2, text_3], 1):
            print(f"\n--- Test {i}: {text[:60]}... ---")
            result = get_defense_data(text, minified_taxonomy)

            if "error" in result:
                print("Error:", result["error"])
            else:
                print(f"Reasoning:              {result.get('_reasoning')}")
                print(f"Market Segment:         {result.get('Market Segment')}")
                print(f"System Type (General):  {result.get('System Type (General)')}")
                print(f"System Type (Specific): {result.get('System Type (Specific)')}")
                print(f"System Name (General):  {result.get('System Name (General)')}")
                print(f"System Name (Specific): {result.get('System Name (Specific)')}")
                print(f"System Piloting:        {result.get('System Piloting')}")


--- Test 1: Palantir USG Inc., Palo Alto, California, has been awarded a... ---
Reasoning:              The text mentions 'Project Brown Heron Data-as-a-Service', which is a specific data service platform. According to the taxonomy, this falls under C4ISR Systems as it is a Data-as-a-Service platform. The system is not a physical platform that operates with a pilot, hence it is categorized as 'Not Applicable' for System Piloting.
Market Segment:         C4ISR Systems
System Type (General):  Other C4ISR
System Type (Specific): Not Applicable
System Name (General):  Project Brown Heron Data-as-a-Service
System Name (Specific): Project Brown Heron Data-as-a-Service
System Piloting:        Not Applicable

--- Test 2: Timken Gears & Services Inc., King of Prussia, Pennsylvania,... ---
Reasoning:              The input text mentions 'DDG-51 Class Hulls', which directly corresponds to the definition of 'Destroyers' in the taxonomy. The specific system name is 'DDG-51', and the general name i

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import os
import json
from openai import OpenAI

In [ ]:
# OpenRouter Configuration
OPENROUTER_API_KEY='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'

client = OpenAI(
    base_url='https://openrouter.ai/api/v1',
    api_key=OPENROUTER_API_KEY
)
client

In [47]:
df_examples = pd.read_excel("/content/Market Segment.xlsx")

In [48]:
df_examples = df_examples.fillna("Not Applicable")

In [49]:
df_examples.head()

,Description of Contract,Market Segment,System Type (General),System Type (Specific),System Name (General),System Name (Specific),System Piloting
0,"Palantir USG Inc., Palo Alto, California, has ...",C4ISR Systems,Other C4ISR,Not Applicable,Project Brown Heron Data-as-a-Service,Project Brown Heron Data-as-a-Service,Not Applicable
1,"Sierra Nevada Corp., Sparks, Nevada, has been ...",Air Platforms,Fixed Wing,Transport Aircraft,C-130,"MC-130J AbMN production kits, spares, interim ...",Crewed
2,"PNH Resources Pte Ltd., Singapore (N40084-22-D...",Infrastructure,Not Applicable,Not Applicable,Construction projects,Construction projects,Not Applicable
3,"PNH Resources Pte Ltd., Singapore (N40084-22-D...",Infrastructure,Not Applicable,Not Applicable,Construction projects,Construction projects,Not Applicable
4,"Timken Gears & Services Inc., King of Prussia,...",Naval Platforms,Surface Combatants,Destroyers,Arleigh Burke-class,USS Arleigh Burke (DDG-51) Class Hulls 138 and...,Crewed


In [50]:
vectorizer = TfidfVectorizer(stop_words='english')
example_vectors = vectorizer.fit_transform(df_examples['Description of Contract'])

In [51]:
def get_similar_example(new_text):
    """
    Finds the single most similar contract from the analyst file.
    Returns the text and the correct classification.
    """
    new_vec = vectorizer.transform([new_text])
    similarities = cosine_similarity(new_vec, example_vectors).flatten()
    best_idx = similarities.argmax()

    # Only use example if it's somewhat similar (score > 0.1)
    if similarities[best_idx] > 0.1:
        row = df_examples.iloc[best_idx]
        return {
            "text": row['Description of Contract'],
            "classification": {
                "Market Segment": row['Market Segment'],
                "System Type (General)": row['System Type (General)'],
                "System Type (Specific)": row['System Type (Specific)'],
                "System Name (General)": row['System Name (General)'],
                "System Name (Specific)": row['System Name (Specific)'],
                "System Piloting": row.get('System Piloting', "Derived from logic")
            }
        }
    return None

In [52]:
def load_taxonomy(file_path="/content/taxonomy.json"):
    with open(file_path, 'r', encoding='utf-8') as f:
        return json.dumps(json.load(f), separators=(',', ':'))

In [53]:
def classify_with_memory(paragraph, taxonomy_string):

    # 1. Find a similar past case
    similar_case = get_similar_example(paragraph)

    # 2. Construct the dynamic prompt
    system_instruction = f"""
    You are a Defense Contract Analyst.
    Your goal is to extract technical data points from the "Input Text".

    REFERENCE TAXONOMY:
    {taxonomy_string}
    """

    user_message = f"Input Text: {paragraph}\n\n"

    # Inject the "One-Shot" Example if found
    if similar_case:
        user_message += f"""
        IMPORTANT REFERENCE - Here is a similar contract classified by a human analyst.
        Use this as a guide for your logic:

        [Past Input]: {similar_case['text'][:300]}...
        [Past Correct Output]: {json.dumps(similar_case['classification'])}

        Now, apply the same logic to the current Input Text.
        """

    # 3. Add Instructions for Naming & Piloting
    user_message += """
    --------------------------------------------------------
    REQUIREMENTS:
    1. Classify 'Market Segment', 'System Type (General)', 'System Type (Specific)' using the Taxonomy.
    2. Extract 'System Name (Specific)' (e.g., MC-130J) and 'System Name (General)' (e.g., C-130).
    3. Determine 'System Piloting' (Crewed, Uncrewed, or Not Applicable).
       - Software/Services/Ammo/Infra = "Not Applicable".
       - Manned Vehicles = "Crewed".
       - Drones/Satellites = "Uncrewed".

    Return JSON only.
    """

    # 4. Call LLM
    completion = client.chat.completions.create(
        model="openai/gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_instruction},
            {"role": "user", "content": user_message}
        ],
        temperature=0,
        response_format={"type": "json_object"}
    )

    return json.loads(completion.choices[0].message.content)

In [60]:
if __name__ == "__main__":
    taxonomy_str = load_taxonomy()

    # Test with a text similar to one in your Excel file
    new_input = "Jacobs/HDR JV, Arlington, Virginia, is awarded a not-to-exceed $90,000,000 indefinite-delivery/indefinite-quantity contract for architect-engineer services for facilities management and sustainment support for Naval Facilities Engineering Systems Command (NAVFAC) worldwide. The work to be performed includes, but is not limited to, facility and utility inventory condition assessment programs; engineering services in support of integrated priority list development; engineering studies, surveys, inspections, and assessments; developing training, handbooks and/or manuals; re-commissioning and retro-commissioning of facilities including facility modifications; and development of facilities management initiatives and processes. Work on this contract will be performed at various Navy and Marine Corps facilities and other government facilities within the NAVFAC Atlantic and Pacific areas of responsibility to include California (5%); Virginia (5%); Florida (5%); North Carolina (5%); South Carolina (5%); Maryland (5%); Washington (5%); Georgia (5%); Hawaii (5%); Rhode Island (5%); Connecticut (5%); Maine (5%); Illinois (5%); Louisiana (5%); Mississippi (5%); Nevada (5 %); Tennessee (5%); Texas (5%); Washington, D.C. (5%); and Naples, Italy (5%). The term of the contract is not to exceed 60 months with an expected completion date of June 2026. No task orders are being awarded at this time. Fiscal 2021 operation and maintenance (Navy) contract funds in the amount of $10,000 are obligated on this award and will expire at the end of the current fiscal year. Future task orders will be primarily funded by operation and maintenance (Navy) contract funds. This contract was competitively procured via the beta.sam.gov website with two proposals received. NAVFAC Atlantic, Norfolk, Virginia, is the contracting activity (N62470-21-D-0008)."

    result = classify_with_memory(new_input, taxonomy_str)

    print("--- Classification Result ---")
    print(json.dumps(result, indent=2))

--- Classification Result ---
{
  "Market Segment": "Infrastructure",
  "System Type (General)": "Other",
  "System Type (Specific)": "Not Applicable",
  "System Name (General)": "Architect-Engineer Services for Facilities Management and Sustainment Support for Naval Facilities Engineering Systems Command (NAVFAC)",
  "System Name (Specific)": "Architect-Engineer Services for Facilities Management and Sustainment Support for Naval Facilities Engineering Systems Command (NAVFAC)",
  "System Piloting": "Not Applicable"
}


In [ ]:
# ==============================================================================
# 1. TAXONOMY STRING (UPDATED FROM EXCEL BOOK3.XLSX)
# ==============================================================================
TAXONOMY_STR = """
[
  {
    "market_segment": "Air Platforms",
    "definition": "Defense aircrafts such as fighter jets, helicopters, and UAVs.",
    "system_types_general": [
      {
        "name": "Fixed Wing",
        "definition": "Aircraft with stationary wings.",
        "system_types_specific": [
          { "name": "Fighter", "definition": "Combat aircraft designed for air-to-air combat." },
          { "name": "Bomber", "definition": "Aircraft designed for dropping bombs." },
          { "name": "Light Combat Aircraft", "definition": "Lightweight multirole jet or turboprop." },
          { "name": "Gunship", "definition": "Heavily armed aircraft for ground support." },
          { "name": "Trainers", "definition": "Aircraft for pilot instruction." },
          { "name": "Tanker", "definition": "Aircraft for in-flight refuelling." },
          { "name": "Maritime Aircraft", "definition": "Aircraft for naval surveillance and strike." },
          { "name": "ISR-Strike", "definition": "Intelligence, Surveillance, Reconnaissance with strike capability." },
          { "name": "C4ISR", "definition": "Airborne Command and Control platforms (e.g., AWACS)." },
          { "name": "AEW&C", "definition": "Airborne Early Warning and Control." },
          { "name": "Transport Aircraft", "definition": "Aircraft used for troop and cargo transport." },
          { "name": "Target Drone", "definition": "Unmanned aerial target for training." },
          { "name": "Fixed Wing Other", "definition": "Miscellaneous fixed-wing platforms." },
          { "name": "Not Applicable", "definition": "Select this when the specific system type is not listed." }
        ]
      },
      {
        "name": "Rotary Wing",
        "definition": "Helicopters and tiltrotors.",
        "system_types_specific": [
          { "name": "Rotary Wing Attack", "definition": "Helicopters for offensive missions." },
          { "name": "Rotary Wing Maritime", "definition": "Naval helicopters." },
          { "name": "Rotary Wing Scout", "definition": "Reconnaissance helicopters." },
          { "name": "Rotary Wing Transport", "definition": "Helicopters for transport." },
          { "name": "Rotary Wing Other", "definition": "Other rotary wing platforms." },
          { "name": "Not Applicable", "definition": "Select this when the specific system type is not listed." }
        ]
      },
      {
        "name": "Other Lift Types",
        "definition": "Non-traditional aircraft.",
        "system_types_specific": [
          { "name": "Hybrid", "definition": "Aircraft with fixed and rotary traits (e.g., V-22)." },
          { "name": "Airship", "definition": "Lighter-than-air craft." },
          { "name": "Parafoil", "definition": "Parachute-based lift systems." },
          { "name": "Not Applicable", "definition": "Select this when the specific system type is not listed." }
        ]
      },
      {
        "name": "Not Applicable",
        "definition": "General Air Platforms.",
        "system_types_specific": [{ "name": "Not Applicable", "definition": "Select this when specific type is not applicable." }]
      }
    ]
  },
  {
    "market_segment": "Land Platforms",
    "definition": "Tanks, armored vehicles, logistic vehicles.",
    "system_types_general": [
      {
        "name": "Artillery",
        "definition": "Large-caliber guns.",
        "system_types_specific": [
          { "name": "Self-Propelled Artillery", "definition": "Mobile artillery vehicles." },
          { "name": "Towed Artillery", "definition": "Stationary artillery moved by other vehicles." },
          { "name": "Mortar", "definition": "Indirect fire weapon." },
          { "name": "MRL (Multiple Rocket Launcher)", "definition": "Rocket artillery systems." },
          { "name": "Other", "definition": "Other artillery systems." },
          { "name": "Not Applicable", "definition": "Select this when the specific system type is not listed." }
        ]
      },
      {
        "name": "Armoured Fighting Vehicles",
        "definition": "Combat vehicles.",
        "system_types_specific": [
          { "name": "Main Battle Tank", "definition": "Heavily armoured front-line tank." },
          { "name": "Tracked Armoured Fighting Vehicles", "definition": "Armoured combat vehicles on tracks." },
          { "name": "Wheeled Armoured Fighting Vehicles", "definition": "Armoured combat vehicles on wheels." },
          { "name": "Amphibious Assault Vehicles", "definition": "Vehicles designed for beach landings." },
          { "name": "Light Tank", "definition": "Lighter weight tank platforms." },
          { "name": "Assault Vehicle", "definition": "Vehicles designed for direct assault." },
          { "name": "Reconnaissance Vehicle", "definition": "Scout vehicles." },
          { "name": "IFV (Infantry Fighting Vehicle)", "definition": "Carries infantry into battle with fire support." },
          { "name": "APC (Armoured Personnel Carrier)", "definition": "Transports infantry." },
          { "name": "AUV (Armoured Utlity Vehicle)", "definition": "Armoured support vehicles." },
          { "name": "Other", "definition": "Other AFVs." },
          { "name": "Not Applicable", "definition": "Select this when the specific system type is not listed." }
        ]
      },
      {
        "name": "Logistics & Support",
        "definition": "Vehicles for support.",
        "system_types_specific": [
          { "name": "All-Terrain Vehicles", "definition": "Light mobility vehicles." },
          { "name": "Combat Engineering Vehicles", "definition": "Vehicles for engineering tasks under fire." },
          { "name": "Construction Vehicles", "definition": "Earthmoving and construction." },
          { "name": "Engineering Equipment", "definition": "Specialized engineering tools." },
          { "name": "Light Tactical/Utility Vehicles", "definition": "Jeeps, light trucks." },
          { "name": "Medical Equipment", "definition": "Ambulances and medical support." },
          { "name": "Military Bridges", "definition": "Bridging layers." },
          { "name": "NBC Equipment", "definition": "Nuclear, Biological, Chemical defense." },
          { "name": "Trucks", "definition": "General transport/resupply." },
          { "name": "Other", "definition": "Other logistics vehicles." },
          { "name": "Not Applicable", "definition": "Select this when the specific system type is not listed." }
        ]
      },
      {
        "name": "Rolling Stock",
        "definition": "Railway systems.",
        "system_types_specific": [
          { "name": "Other", "definition": "Railway platforms." },
          { "name": "Not Applicable", "definition": "Not applicable." }
        ]
      },
      {
        "name": "Other Land",
        "definition": "Miscellaneous land systems.",
        "system_types_specific": [
          { "name": "Soldier Fighting Systems", "definition": "Individual soldier gear and tech." },
          { "name": "Not Applicable", "definition": "Not applicable." }
        ]
      },
      {
        "name": "Not Applicable",
        "definition": "General Land Platforms.",
        "system_types_specific": [{ "name": "Not Applicable", "definition": "Select this when specific type is not applicable." }]
      }
    ]
  },
  {
    "market_segment": "Naval Platforms",
    "definition": "Military vessels, ships, submarines.",
    "system_types_general": [
      {
        "name": "Sub-Surface",
        "definition": "Submarines.",
        "system_types_specific": [
          { "name": "Diesel-Powered Submarine", "definition": "Conventional submarines." },
          { "name": "Nuclear-Powered Submarine", "definition": "Nuclear propulsion submarines." },
          { "name": "Other", "definition": "Other sub-surface vessels." },
          { "name": "Not Applicable", "definition": "Select this when the specific system type is not listed." }
        ]
      },
      {
        "name": "Surface Combatants",
        "definition": "Warships.",
        "system_types_specific": [
          { "name": "Aircraft Carrier", "definition": "Ship for launching aircraft." },
          { "name": "Cruisers", "definition": "Large multi-role warships." },
          { "name": "Destroyers", "definition": "Fast, maneuverable long-endurance warships." },
          { "name": "Frigates", "definition": "Warships sized between destroyer and corvette." },
          { "name": "Corvettes", "definition": "Small warships." },
          { "name": "Other Surface Combatants", "definition": "Other surface fighting ships." },
          { "name": "Not Applicable", "definition": "Select this when the specific system type is not listed." }
        ]
      },
      {
        "name": "Patrol and Costal Combatants",
        "definition": "Coastal defense vessels.",
        "system_types_specific": [
          { "name": "Patrol Boat/Craft - Coastal", "definition": "Short range patrol." },
          { "name": "Patrol Boat/Craft - Ocean", "definition": "Long range patrol." },
          { "name": "Patrol Boat/Craft - Riverine", "definition": "River patrol." },
          { "name": "Not Applicable", "definition": "Select this when the specific system type is not listed." }
        ]
      },
      {
        "name": "Mine Warfare/Mine Countermeasures (Maritime)",
        "definition": "Mine detection/removal.",
        "system_types_specific": [
          { "name": "Mine Sweeper", "definition": "Detects and clears mines." },
          { "name": "Mine Counter-Measures", "definition": "Systems to neutralize mines." },
          { "name": "Mine Hunter", "definition": "Actively hunts mines." },
          { "name": "Not Applicable", "definition": "Select this when the specific system type is not listed." }
        ]
      },
      {
        "name": "Amphibious (Maritime)",
        "definition": "Ships for amphibious assault.",
        "system_types_specific": [
          { "name": "Amphibious Assault Ship", "definition": "Supports landings (LHD/LHA)." },
          { "name": "Landing Ship", "definition": "Transports vehicles/troops to shore." },
          { "name": "Landing Craft", "definition": "Small craft for beach landing." },
          { "name": "Landing Platform Dock", "definition": "LPD class ships." },
          { "name": "Amphibious Other (Maritime)", "definition": "Other amphibious vessels." },
          { "name": "Not Applicable", "definition": "Select this when the specific system type is not listed." }
        ]
      },
      {
        "name": "Logistics and Support (Maritime)",
        "definition": "Support vessels.",
        "system_types_specific": [
          { "name": "Research/Survey Vessels", "definition": "Oceanographic ships." },
          { "name": "Icebreakers", "definition": "Polar region ships." },
          { "name": "Combat Support Ship", "definition": "Replenishment and support." },
          { "name": "Repair Ships", "definition": "Maintenance at sea." },
          { "name": "Ocean Transport/Tanker/Cargo/Oiler", "definition": "Fuel and cargo transport." },
          { "name": "Hospital Ship", "definition": "Medical facilities at sea." },
          { "name": "Logistics and Support Other (Maritime)", "definition": "Other support." },
          { "name": "Not Applicable", "definition": "Select this when the specific system type is not listed." }
        ]
      },
      {
        "name": "Other Maritime",
        "definition": "Miscellaneous naval.",
        "system_types_specific": [
          { "name": "Training Ship", "definition": "Vessels for training sailors." },
          { "name": "Other Maritime", "definition": "Other." },
          { "name": "Not Applicable", "definition": "Select this when the specific system type is not listed." }
        ]
      },
      {
        "name": "Not Applicable",
        "definition": "General Naval Platforms.",
        "system_types_specific": [{ "name": "Not Applicable", "definition": "Select this when specific type is not applicable." }]
      }
    ]
  },
  {
    "market_segment": "Space Systems",
    "definition": "Weapons or platforms based on space technologies.",
    "system_types_general": [
      {
        "name": "Launch Vehicle",
        "definition": "Launch systems.",
        "system_types_specific": [
          { "name": "Launch Vehicle", "definition": "Rockets for space launch." },
          { "name": "Not Applicable", "definition": "Not applicable." }
        ]
      },
      {
        "name": "Satellite",
        "definition": "Orbiting objects.",
        "system_types_specific": [
          { "name": "Satellite", "definition": "General satellites." },
          { "name": "Radar Satellite", "definition": "SAR/Radar equipped satellites." },
          { "name": "Not Applicable", "definition": "Not applicable." }
        ]
      },
      {
        "name": "Anti-Space",
        "definition": "Systems to deny space usage.",
        "system_types_specific": [
          { "name": "Anti-Space", "definition": "ASAT systems." },
          { "name": "Not Applicable", "definition": "Not applicable." }
        ]
      },
      {
        "name": "Other",
        "definition": "Other space systems.",
        "system_types_specific": [
          { "name": "Other", "definition": "Other." },
          { "name": "Not Applicable", "definition": "Not applicable." }
        ]
      },
      {
        "name": "Not Applicable",
        "definition": "General Space Systems.",
        "system_types_specific": [
          { "name": "Not Applicable", "definition": "Not applicable." }
        ]
      }
    ]
  },
  {
    "market_segment": "C4ISR Systems",
    "definition": "Command, Control, Comms, Intel, Surveillance.",
    "system_types_general": [
      {
        "name": "Electro-optic Sensor",
        "definition": "Optical systems.",
        "system_types_specific": [
          { "name": "Imaging EO/IR", "definition": "Cameras and Infrared sensors." },
          { "name": "Targeting EO/IR", "definition": "Used for weapon guidance." },
          { "name": "Not Applicable", "definition": "Select this when the specific system type is not listed." }
        ]
      },
      {
        "name": "Radar",
        "definition": "Radar systems.",
        "system_types_specific": [
          { "name": "Air Search Radar", "definition": "Detects airborne targets." },
          { "name": "Surface Surveillance Radar", "definition": "Detects surface targets." },
          { "name": "Fire Control Radar", "definition": "Directs weapons." },
          { "name": "Navigation Radar", "definition": "For navigation." },
          { "name": "Weather Radar", "definition": "For weather." },
          { "name": "Multi-Mission Radar", "definition": "Versatile radar systems." },
          { "name": "Not Applicable", "definition": "Select this when the specific system type is not listed." }
        ]
      },
      {
        "name": "Sonar",
        "definition": "Underwater detection.",
        "system_types_specific": [
          { "name": "Sonar", "definition": "Sonar systems." },
          { "name": "Not Applicable", "definition": "Not applicable." }
        ]
      },
      {
        "name": "Command and Control",
        "definition": "C2 systems.",
        "system_types_specific": [
          { "name": "Command and Control System", "definition": "C2 software and hardware." },
          { "name": "Not Applicable", "definition": "Not applicable." }
        ]
      },
      {
        "name": "Communications",
        "definition": "Comms gear.",
        "system_types_specific": [
          { "name": "Radios", "definition": "Tactical radios." },
          { "name": "Data links", "definition": "Tactical data links." },
          { "name": "Satellite Communications", "definition": "SATCOM." },
          { "name": "Not Applicable", "definition": "Select this when the specific system type is not listed." }
        ]
      },
      {
        "name": "Computers",
        "definition": "Computing hardware.",
        "system_types_specific": [
          { "name": "Military IT Systems", "definition": "Ruggedized or mil-spec IT." },
          { "name": "Not Applicable", "definition": "Not applicable." }
        ]
      },
      {
        "name": "Cyber",
        "definition": "Cyber warfare.",
        "system_types_specific": [
          { "name": "Cyber Defense", "definition": "Protective cyber systems." },
          { "name": "Cyber Offense", "definition": "Offensive cyber capabilities." },
          { "name": "Not Applicable", "definition": "Not applicable." }
        ]
      },
      {
        "name": "Artificial Intelligence",
        "definition": "AI systems.",
        "system_types_specific": [
          { "name": "Artificial Intelligence", "definition": "AI/ML software." },
          { "name": "Not Applicable", "definition": "Not applicable." }
        ]
      },
      {
        "name": "Other C4ISR",
        "definition": "Miscellaneous C4ISR.",
        "system_types_specific": [
          { "name": "PNT Systems", "definition": "Positioning, Navigation, and Timing." },
          { "name": "Other", "definition": "Other." },
          { "name": "Not Applicable", "definition": "Not applicable." }
        ]
      },
      {
        "name": "Integrated C4ISR System",
        "definition": "Combined C4ISR.",
        "system_types_specific": [
          { "name": "Integrated C4ISR System", "definition": "System of systems." },
          { "name": "Not Applicable", "definition": "Not applicable." }
        ]
      },
      {
        "name": "Defensive Systems",
        "definition": "Countermeasures.",
        "system_types_specific": [
          { "name": "Defensive Aid Suite", "definition": "Integrated defensive system." },
          { "name": "Not Applicable", "definition": "Select this when the specific system type is not listed." }
        ]
      },
      {
        "name": "Not Applicable",
        "definition": "General C4ISR.",
        "system_types_specific": [{ "name": "Not Applicable", "definition": "Select this when specific type is not applicable." }]
      }
    ]
  },
  {
    "market_segment": "Weapon Systems",
    "definition": "Missiles, bombs, guns.",
    "system_types_general": [
      {
        "name": "Missile",
        "definition": "Guided weapons (powered).",
        "system_types_specific": [
          { "name": "Air-to-Air", "definition": "AAM." },
          { "name": "Air-to-Surface", "definition": "ASM." },
          { "name": "Air-to-Ship", "definition": "Anti-ship missile." },
          { "name": "Surface-to-Air", "definition": "SAM." },
          { "name": "Surface-to-Surface", "definition": "SSM." },
          { "name": "Surface-to-Ship", "definition": "Coastal defense missile." },
          { "name": "Ship-to-Air", "definition": "Naval SAM." },
          { "name": "Ship-to-Surface", "definition": "Naval strike missile." },
          { "name": "Ship-to-Ship", "definition": "Naval anti-ship missile." },
          { "name": "Ballistic Missile", "definition": "Rocket-propelled strategic-weapons." },
          { "name": "Other", "definition": "Other missile types." },
          { "name": "Not Applicable", "definition": "Select this when the specific system type is not listed." }
        ]
      },
      {
        "name": "Precision Guided Weapons",
        "definition": "Smart munitions.",
        "system_types_specific": [
          { "name": "Guided Rocket", "definition": "Guided unpowered or short burn rockets." },
          { "name": "Guided Bomb", "definition": "Smart bombs (JDAM etc)." },
          { "name": "Guided Artillery Round", "definition": "Smart shells." },
          { "name": "Loitering Munitions", "definition": "Suicide drones." },
          { "name": "Other", "definition": "Other PGMs." },
          { "name": "Not Applicable", "definition": "Select this when the specific system type is not listed." }
        ]
      },
      {
        "name": "Torpedo",
        "definition": "Underwater missiles.",
        "system_types_specific": [
          { "name": "Heavy Weight Torpedo", "definition": "Submarine launched torpedo." },
          { "name": "Light Weight Torpedo", "definition": "Air/Surface launched torpedo." },
          { "name": "Other", "definition": "Other." },
          { "name": "Not Applicable", "definition": "Select this when the specific system type is not listed." }
        ]
      },
      {
        "name": "Directed Energy Weapon",
        "definition": "Lasers and high-power microwaves.",
        "system_types_specific": [
          { "name": "Radar", "definition": "High power radar as weapon." },
          { "name": "Laser", "definition": "High energy laser." },
          { "name": "Microwave", "definition": "HPM weapons." },
          { "name": "Sonic", "definition": "Acoustic weapons." },
          { "name": "Not Applicable", "definition": "Select this when the specific system type is not listed." }
        ]
      },
      {
        "name": "Integrated Weapon Systems",
        "definition": "Combined weapon platforms.",
        "system_types_specific": [
          { "name": "Air and Missile Defense System", "definition": "Integrated AMD." },
          { "name": "Ballistic Missile Defense System", "definition": "BMD." },
          { "name": "Coastal Defense System", "definition": "Coastal protection batteries." },
          { "name": "Border Defense System", "definition": "Border security weapons." },
          { "name": "Other", "definition": "Other." },
          { "name": "Not Applicable", "definition": "Select this when the specific system type is not applicable." }
        ]
      },
      {
        "name": "Swarm Drones",
        "definition": "Drone swarms.",
        "system_types_specific": [
          { "name": "EW Swarm Drones", "definition": "Swarms for electronic warfare." },
          { "name": "Kinetic Strike Swarm Drones", "definition": "Swarms for attack." },
          { "name": "Other", "definition": "Other." },
          { "name": "Not Applicable", "definition": "Not applicable." }
        ]
      },
      {
        "name": "Launchers",
        "definition": "Weapon firing systems.",
        "system_types_specific": [
          { "name": "Launchers", "definition": "Missile or rocket launchers." },
          { "name": "Remote Weapon System", "definition": "RWS." },
          { "name": "Not Applicable", "definition": "Not applicable." }
        ]
      },
      {
        "name": "Mines",
        "definition": "Explosive traps.",
        "system_types_specific": [
          { "name": "Anti-Personnel Mine", "definition": "Targeting foot traffic." },
          { "name": "Anti-Tank Mine", "definition": "Targeting vehicles." },
          { "name": "Sea Mine", "definition": "Targeting ships." },
          { "name": "Mine Dispersal System", "definition": "Systems to scatter mines." },
          { "name": "Not Applicable", "definition": "Not applicable." }
        ]
      },
      {
        "name": "Small Arms & Ammunition",
        "definition": "Guns and bullets.",
        "system_types_specific": [
          { "name": "Small Arms", "definition": "Rifles, pistols, machine guns." },
          { "name": "Ammunition", "definition": "Bullets and shells." },
          { "name": "Rockets", "definition": "Unguided rockets." },
          { "name": "Bombs", "definition": "Unguided bombs." },
          { "name": "Explosives", "definition": "C4, TNT, etc." },
          { "name": "Not Applicable", "definition": "Not applicable." }
        ]
      },
      {
        "name": "Electronic Warfare Systems",
        "definition": "EW Systems.",
        "system_types_specific": [
          { "name": "Electronic Jamming", "definition": "Jamming systems." },
          { "name": "Electronic Attack", "definition": "Offensive EW." },
          { "name": "Other", "definition": "Other EW." },
          { "name": "Not Applicable", "definition": "Select this when the specific system type is not listed." }
        ]
      },
      {
        "name": "Not Applicable",
        "definition": "General Weapon Systems.",
        "system_types_specific": [{ "name": "Not Applicable", "definition": "Select this when specific type is not applicable." }]
      }
    ]
  },
  {
    "market_segment": "Infrastructure",
    "definition": "Military buildings and facilities.",
    "system_types_general": [
      {
        "name": "Shipyards/Ports/Harbours",
        "definition": "Maritime infrastructure.",
        "system_types_specific": [{ "name": "Shipyards/Ports/Harbours", "definition": "Shipyards/Ports/Harbours" }, { "name": "Not Applicable", "definition": "N/A" }]
      },
      {
        "name": "Aircraft Basing",
        "definition": "Air infrastructure.",
        "system_types_specific": [{ "name": "Aircraft Basing", "definition": "Aircraft Basing" }, { "name": "Not Applicable", "definition": "N/A" }]
      },
      {
        "name": "Accomodation",
        "definition": "Housing.",
        "system_types_specific": [{ "name": "Accomodation", "definition": "Accomodation" }, { "name": "Not Applicable", "definition": "N/A" }]
      },
      {
        "name": "Non-military IT",
        "definition": "Office IT.",
        "system_types_specific": [{ "name": "Non-military IT", "definition": "Non-military IT" }, { "name": "Not Applicable", "definition": "N/A" }]
      },
      {
        "name": "Land",
        "definition": "Real estate.",
        "system_types_specific": [{ "name": "Land", "definition": "Land" }, { "name": "Not Applicable", "definition": "N/A" }]
      },
      {
        "name": "Training Facilities",
        "definition": "Schools and ranges.",
        "system_types_specific": [{ "name": "Training Facilities", "definition": "Training Facilities" }, { "name": "Not Applicable", "definition": "N/A" }]
      },
      {
        "name": "Air Traffic Management",
        "definition": "ATC infra.",
        "system_types_specific": [{ "name": "Air Traffic Management", "definition": "Air Traffic Management" }, { "name": "Not Applicable", "definition": "N/A" }]
      },
      {
        "name": "RDT&E Facilities",
        "definition": "Research labs.",
        "system_types_specific": [{ "name": "RDT&E Facilities", "definition": "RDT&E Facilities" }, { "name": "Not Applicable", "definition": "N/A" }]
      },
      {
        "name": "Production Facilities",
        "definition": "Factories.",
        "system_types_specific": [{ "name": "Production Facilities", "definition": "Production Facilities" }, { "name": "Not Applicable", "definition": "N/A" }]
      },
      {
        "name": "Maintenance Facilities",
        "definition": "Repair and maintenance buildings.",
        "system_types_specific": [{ "name": "Maintenance Facilities", "definition": "Maintenance Facilities" }, { "name": "Not Applicable", "definition": "N/A" }]
      },
      {
        "name": "Utilities",
        "definition": "Power, water, waste.",
        "system_types_specific": [{ "name": "Utilities", "definition": "Utilities" }, { "name": "Not Applicable", "definition": "N/A" }]
      },
      {
        "name": "Other",
        "definition": "Other infrastructure.",
        "system_types_specific": [{ "name": "Other", "definition": "Other" }, { "name": "Not Applicable", "definition": "N/A" }]
      },
      {
        "name": "Not Applicable",
        "definition": "General construction.",
        "system_types_specific": [
          { "name": "Not Applicable", "definition": "N/A" }
        ]
      }
    ]
  },
  {
    "market_segment": "Training & Simulation",
    "definition": "Training services and equipment.",
    "system_types_general": [
      {
        "name": "Simulators",
        "definition": "Simulation hardware/software.",
        "system_types_specific": [
          { "name": "Simulators", "definition": "Simulators." },
          { "name": "Not Applicable", "definition": "N/A" }
        ]
      },
      {
        "name": "Training Aids",
        "definition": "Tools for training.",
        "system_types_specific": [
          { "name": "Training Aids", "definition": "Training Aids." },
          { "name": "Not Applicable", "definition": "N/A" }
        ]
      },
      {
        "name": "Other",
        "definition": "Other training.",
        "system_types_specific": [
          { "name": "Other", "definition": "Other." },
          { "name": "Not Applicable", "definition": "N/A" }
        ]
      },
      {
        "name": "Not Applicable",
        "definition": "General Training.",
        "system_types_specific": [{ "name": "Not Applicable", "definition": "Select this when specific type is not applicable." }]
      }
    ]
  },
  {
    "market_segment": "Unknown",
    "definition": "Unknown segment.",
    "system_types_general": [
      {
        "name": "Not Applicable",
        "definition": "Select this when the system type is not applicable.",
        "system_types_specific": [{ "name": "Not Applicable", "definition": "Select this when the specific system type is not applicable." }]
      }
    ]
  }
]
"""

# ==============================================================================
# 2. CONSTANTS (EXISTING)
# ==============================================================================

VALID_DEPENDENCIES = {
    "Air Platforms": [
        "Fixed Wing", "Rotary Wing", "Other Lift Types", "Not Applicable"
    ],
    "Land Platforms": [
        "Artillery", "Armoured Fighting Vehicles", "Logistics & Support",
        "Rolling Stock", "Other Land", "Not Applicable"
    ],
    "Naval Platforms": [
        "Sub-Surface", "Surface Combatants", "Patrol and Costal Combatants",
        "Mine Warfare/Mine Countermeasures (Maritime)", "Amphibious (Maritime)",
        "Logistics and Support (Maritime)", "Other Maritime", "Not Applicable"
    ],
    "Space Systems": [
        "Launch Vehicle", "Satellite", "Anti-Space", "Other", "Not Applicable"
    ],
    "C4ISR Systems": [
        "Electro-optic Sensor", "Radar", "Sonar", "Command and Control",
        "Communications", "Computers", "Cyber", "Artificial Intelligence",
        "Other C4ISR", "Integrated C4ISR System", "Defensive Systems", "Not Applicable"
    ],
    "Weapon Systems": [
        "Missile", "Precision Guided Weapons", "Torpedo", "Directed Energy Weapon",
        "Integrated Weapon Systems", "Swarm Drones", "Launchers", "Mines",
        "Small Arms & Ammunition", "Electronic Warfare Systems", "Not Applicable"
    ],
    "Infrastructure": [
        "Shipyards/Ports/Harbours", "Aircraft Basing", "Accomodation", "Non-military IT",
        "Land", "Training Facilities", "Air Traffic Management", "RDT&E Facilities",
        "Production Facilities", "Maintenance Facilities", "Utilities", "Other", "Not Applicable"
    ],
    "Training & Simulation": ["Simulators", "Training Aids", "Other", "Not Applicable"],
    "Unknown": ["Not Applicable"]
}

# ==============================================================================
# 3. TAXONOMY PROMPT (FOR MARKET SEGMENTATION)
# ==============================================================================
TAXONOMY_PROMPT = """
You are an expert Defense Market Analyst. Your task is to classify defense contracts into a strict three-level taxonomy based on the text description provided.

## Input
Description: {description}

## Taxonomy Logic
1. **Market Segment**: Identify the broad domain (e.g., Air Platforms, Weapon Systems).
2. **System Type (General)**: Based *only* on the selected Market Segment, choose the General category that fits best.
   - Example: If Segment is "Air Platforms", "Fighter" is NOT a General type; "Fixed Wing" IS.
3. **System Type (Specific)**: Based *only* on the selected General type, choose the Specific category.
   - Example: If General is "Fixed Wing", "Fighter" is valid.

## Rules
- Use "Not Applicable" if a specific level is not mentioned or unclear.
- Do not invent new categories. You must strictly use the provided JSON taxonomy structure.
- **Hierarchy Enforcement**: You cannot select a Specific type that does not belong to the selected General type.

## Output Format (JSON)
{
  "market_segment": "String",
  "system_type_general": "String",
  "system_type_specific": "String",
  "reasoning": "Brief explanation."
}
"""

# ==============================================================================
# 4. GOLD STANDARD EXAMPLES (GROUND TRUTH)
# ==============================================================================

GOLD_STANDARD_EXAMPLES = """
### EXAMPLES OF PERFECT EXTRACTION (DO NOT DEVIATE):

--- EXAMPLE 1: SERVICE ON A PLATFORM (E-2D CASE) ---
Input Text:
"Northrop Grumman... modification... to extend services and adds hours increasing the full-scale fatigue repair time... in support of E-2D Advanced Hawkeye aircraft development."

Correct Output:
{{
  "Supplier Name": "Northrop Grumman Aerospace",
  "Program Type": "Procurement",
  "Quantity": "Not Applicable",
  "Value Certainty": "Confirmed",
  "Value (Million)": "12.015"
}}

--- EXAMPLE 2: IT / AGREEMENT (DOD ESI CASE) ---
Input Text:
"Dell Marketing... awarded... under the Department of Defense Enterprise Software Initiative (DOD ESI)... This DOD Enterprise Software Agreement (ESA) will provide Microsoft 365..."

Correct Output:
{{
  "Supplier Name": "Dell Inc",
  "Program Type": "Procurement",
  "Quantity": "Not Applicable",
  "Value Certainty": "Estimated",
  "Value (Million)": "2493.000"
}}

--- EXAMPLE 3: COMPONENT PROCUREMENT (SUBMARINE) ---
Input Text:
"BAE Systems... awarded... for Virginia-class submarine propulsors."

Correct Output:
{{
  "Supplier Name": "BAE Systems",
  "Program Type": "Procurement",
  "Quantity": "Unknown",
  "Value Certainty": "Confirmed",
  "Value (Million)": "18.694"
}}

--- EXAMPLE 4: R&D / PROGRAM (GENERIC) ---
Input Text:
"Georgia Tech... contract... for the development and application of advanced technology solutions."

Correct Output:
{{
  "Supplier Name": "Georgia Tech",
  "Program Type": "RDT&E",
  "Quantity": "Not Applicable",
  "Value Certainty": "Confirmed",
  "Value (Million)": "92.000"
}}

--- EXAMPLE 5: NAVAL LOGISTICS (EPF CASE) ---
Input Text:
"Austal USA... contract modification... for the detail design... and demonstration of autonomous capability in Expeditionary Fast Transport (EPF) 13."

Correct Output:
{{
  "Supplier Name": "Austal Limited",
  "Program Type": "Procurement",
  "Quantity": "Not Applicable",
  "Value Certainty": "Confirmed",
  "Value (Million)": "44.000"
}}

--- EXAMPLE 6: SERVICES (CONSULTATION) ---
Input Text:
"Bowhead Cybersecurity... contract for senior consultation support services."

Correct Output:
{{
  "Supplier Name": "Bowhead",
  "Program Type": "Other Service",
  "Quantity": "Not Applicable",
  "Value Certainty": "Confirmed",
  "Value (Million)": "92.308"
}}

--- EXAMPLE 7: MISSILES (QUANTITY & VARIANTS) ---
Input Text:
"Raytheon Missile and Defense... production and delivery of... 483 AIM-9X Block II... 82 AIM-9X block II plus... 156 Block II Captive Air Training Missiles..."

Correct Output:
{{
  "Supplier Name": "Raytheon Missiles and Defense",
  "Program Type": "Procurement",
  "Quantity": "721",
  "Value Certainty": "Confirmed",
  "Value (Million)": "328.156"
}}

--- EXAMPLE 8: COMPONENT UPGRADE (TANK) ---
Input Text:
"DRS Network & Imaging Systems... contract to procure Advanced Multi-Purpose Round Gunner's Auxiliary Sight units... for M1A2 Abrams."

Correct Output:
{{
  "Supplier Name": "DRS Network and Imaging Systems",
  "Program Type": "Procurement",
  "Quantity": "Unknown",
  "Value Certainty": "Confirmed",
  "Value (Million)": "82.741"
}}

--- EXAMPLE 9: SERVICES ON A SYSTEM (RADAR ENGINEERING) ---
Input Text:
"Lockheed Martin... for engineering services in support of the AN/SPY-1 Elevated Radar Advanced Calibration Experiment."

Correct Output:
{{
  "Supplier Name": "Lockheed Martin – Rotary and Mission Systems",
  "Program Type": "RDT&E",
  "Quantity": "Not Applicable",
  "Value Certainty": "Confirmed",
  "Value (Million)": "8.256"
}}

--- EXAMPLE 10: HELICOPTER SUPPORT ---
Input Text:
"DigiFlight Inc... modification... for programmatic support for the Apache Attack Helicopter."

Correct Output:
{{
  "Supplier Name": "Unknown",
  "Program Type": "Other Service",
  "Quantity": "Not Applicable",
  "Value Certainty": "Confirmed",
  "Value (Million)": "13.803"
}}

--- EXAMPLE 11: DEVELOPMENT vs MRO (EDGE CASE) ---
Input Text:
"Northrop Grumman... modification... to extend services and adds hours increasing the full-scale fatigue repair time... in support of E-2D Advanced Hawkeye aircraft development."

Correct Output:
{{
  "Supplier Name": "Northrop Grumman",
  "Program Type": "Procurement",
  "Quantity": "Not Applicable",
  "Value Certainty": "Confirmed",
  "Value (Million)": "12.015"
}}
"""

# ==============================================================================
# 5. GEOGRAPHY PROMPT
# ==============================================================================
GEOGRAPHY_PROMPT = """
You are a defence contract geography analyst.
YOUR TASK: Extract Customer and Supplier locations and Customer Operator.

STRICT RULES:
1. **Supplier Country**: The country where the supplier company is BASED (not necessarily HQ).
2. **Special Circumstance - Ukraine**:
   - If a country buys equipment UNILATERALLY for Ukraine -> Customer: [Purchasing Country], Operator: "Ukraine (Assistance)".

3. **CUSTOMER OPERATOR (CRITICAL - NO HALLUCINATION):**
   - You MUST pick one value from the provided list ONLY. 
   - If the operator in the text is "Naval Information Warfare Center", map it to "Navy".
   - If "Air Force Life Cycle Management Center", map it to "Air Force".
   - **VALID LIST:** {operators}

MAPPING:
{geo_mapping}

Return JSON ONLY:
{{
  "Customer Region": "...",
  "Customer Country": "...",
  "Customer Operator": "...",
  "Supplier Region": "...",
  "Supplier Country": "..."
}}

Description:
\"\"\"
{text}
\"\"\"
"""

# ==============================================================================
# 6. FINANCIAL PROMPT (UPDATED WITH REFINED DEFINITIONS)
# ==============================================================================
FINANCIAL_PROMPT = f"""
You are a defence contract financial and program analyst.
YOUR TASK: Extract supplier, program info, and financials.

{GOLD_STANDARD_EXAMPLES}

STRICT RULES:

1. **Program Type (CHOOSE EXACTLY ONE)**:
   - **Training**: Purchase of military training *services*. Note: Purchase of training aircraft or simulators falls under "Procurement".
   - **Procurement**: Acquisition of new products, systems, or production kits. **IMPORTANT**: Includes services, repairs, or modifications performed on *test articles* or prototypes to support development/production (e.g., "fatigue repair" for design validation).
   - **MRO/Support**: Maintenance, Repair, and Operations. Select ONLY for contracts related to the sustainment/repair of *existing, fielded, operational* equipment. Do NOT select this if the "repair" is part of a development or testing phase.
   - **RDT&E**: Contracts primarily for research, prototyping, or testing where the outcome is knowledge/design validation rather than a fielded product.
   - **Upgrade**: Purchase of components/services to modernize existing equipment.
   - **Other Service**: General consulting, IT support, or services not tied to a specific weapon system's lifecycle.
   - **Unknown**: If the program type cannot be determined.

2. **Supplier Name (CRITICAL)**: 
   - **Context Understanding**: Read the entire description. Identify the specific entity that has been AWARDED the contract/is performing the work.
   - **Position Independent**: The supplier is usually at the start, but if mentioned later, prioritize the entity actually performing the scope of work.
   - **Taxonomy Alignment**: Output the **Clean Brand Name** (e.g., "The Boeing Co., St. Louis" -> "Boeing").
     - Your goal is to output a name that matches a standard defence taxonomy.
     - Remove legal suffixes ("Inc", "LLC", "Corp") and location details.
     - *Example:* "Lockheed Martin Aeronautics Co." -> "Lockheed Martin".
   - **Fallback**: If the specific supplier is unclear, extract the first company name mentioned in the text.

3. **Quantity (Crucial)**:
   - **Hardware/Missiles:** EXTRACT the total count. If text says "483 AIM-9X... 82 AIM-9X...", SUM THEM UP (483+82=565).
   - **Services/RDT&E/IT:** Use "Not Applicable".
   - **Unknown:** If hardware is bought but no number is given, use "Unknown".

4. **Value Calculation**:
   - Convert to **MILLIONS**.
   - Round to **3 decimal places**.
   - Do NOT include currency symbols or text.
   - Ex: $2,493,000,000 -> "2493.000"

5. **Value Certainty (LOGIC CHANGE)**:
   - **Confirmed**: This is the **DEFAULT**. Use "Confirmed" for all definitive contract awards, modifications, BPAs, and IDIQs where a specific value (or ceiling) is stated.
     - *CRITICAL RULE:* If the text says "estimated value", "ceiling", or "maximum value" in the context of a signed agreement (like a BPA), treat this as **Confirmed** (as it represents the confirmed contract capacity).
   - **Estimated**: Use ONLY if the text explicitly states the value is "potential", "approximate" (without a precise figure), or "projected" outside of the current award structure.

6. **Description Date Found**:
   - For MRO contracts, extract the completion date. Otherwise leave empty.

Return JSON ONLY:
{{{{
  "Supplier Name": "...",
  "Program Type": "...",
  "Quantity": "...",
  "Value Certainty": "...",
  "Value (Million)": "...", 
  "Currency": "...",
  "Description Date Found": "..."
}}}}

Description:
\"\"\"
{{text}}
\"\"\"
"""

# ==============================================================================
# 7. DOMESTIC CONTENT PROMPT
# ==============================================================================
DOMESTIC_CONTENT_PROMPT = """
You are a defence procurement analyst.
YOUR TASK: Classify the "Domestic Content" based on the text.

STRICT DEFINITIONS:
1. Imported: Product originates from a different country and is physically imported.
2. Indigenous: Product is produced within the customer's country.
   - NOTE: This INCLUDES local production units/subsidiaries of a foreign company located in the customer's country.
3. Local Assembly: Components manufactured abroad, imported, and assembled locally (CKD/SKD).
4. License Production: Local company manufactures a foreign product under a licensing agreement.

INPUT CONTEXT:
- Supplier Country: {supplier_country}
- Customer Country: {customer_country}

OPTIONS:
{options}

Return JSON ONLY:
{{
  "Domestic Content": "..."
}}

Description:
\"\"\"
{text}
\"\"\"
"""

GEOGRAPHY_MAPPING = {
    "Sub-Saharan Africa": [
        "Angola", "Benin", "Botswana", "Burkina Faso", "Burundi", "Cameroon", "Cape Verde",
        "Central African Republic", "Chad", "Congo, Democratic Republic of", "Congo, Republic of",
        "Djibouti", "Equatorial Guinea", "Eritrea", "Eswatini", "Ethiopia", "Gabon", "Gambia",
        "Ghana", "Guinea", "Guinea-Bissau", "Ivory Coast", "Kenya", "Lesotho", "Liberia",
        "Madagascar", "Malawi", "Mali", "Mauritius", "Mozambique", "Namibia", "Niger",
        "Nigeria", "Rwanda", "Senegal", "Seychelles", "Sierra Leone", "Somalia", "South Africa",
        "South Sudan", "Sudan", "Tanzania", "Togo", "Uganda", "Zambia", "Zimbabwe"
    ],
    "Asia-Pacific": [
        "Australia", "Brunei", "Cambodia", "China", "Hong Kong", "Indonesia", "Japan", "Laos",
        "Malaysia", "Mongolia", "Myanmar", "New Zealand", "North Korea", "Papua New Guinea",
        "Philippines", "Singapore", "South Korea", "Taiwan", "Thailand", "Vietnam"
    ],
    "Europe": [
        "Albania", "Austria", "Belgium", "Bosnia and Herzegovina", "Bulgaria", "Croatia", "Cyprus",
        "Czech Republic", "Denmark", "Estonia", "Finland", "France", "Georgia", "Germany", "Greece",
        "Hungary", "Iceland", "Ireland", "Italy", "Kosovo", "Latvia", "Lithuania", "Luxembourg",
        "Malta", "Montenegro", "Netherlands", "North Macedonia", "Norway", "Poland", "Portugal",
        "Romania", "Serbia", "Slovakia", "Slovenia", "Spain", "Sweden", "Switzerland", "Turkey",
        "Ukraine", "United Kingdom"
    ],
    "Latin America": [
        "Argentina", "Bahamas", "Barbados", "Belize", "Bolivia", "Brazil", "Chile", "Colombia",
        "Costa Rica", "Cuba", "Curacao", "Dominican Republic", "Ecuador", "El Salvador", "Guatemala",
        "Guyana", "Haiti", "Honduras", "Jamaica", "Mexico", "Nicaragua", "Panama", "Paraguay",
        "Peru", "Suriname", "Trinidad and Tobago", "Uruguay", "Venezuela"
    ],
    "Middle East and North Africa": [
        "Algeria", "Bahrain", "Egypt", "Iran", "Iraq", "Israel", "Jordan", "Kuwait", "Lebanon",
        "Libya", "Mauritania", "Morocco", "Oman", "Qatar", "Saudi Arabia", "Syria", "Tunisia",
        "United Arab Emirates", "Yemen"
    ],
    "North America": ["Canada", "USA"],
    "Russia & CIS": [
        "Armenia", "Azerbaijan", "Belarus", "Kazakhstan", "Kyrgyzstan", "Moldova", "Russia",
        "Tajikistan", "Turkmenistan", "Uzbekistan"
    ],
    "South Asia": [
        "Afghanistan", "Bangladesh", "India", "Maldives", "Nepal", "Pakistan", "Sri Lanka"
    ],
    "Unknown": [
        "Andorra", "Antigua and Barbuda", "Bhutan", "Comoros", "Dominica", "Federated States of Micronesia",
        "Fiji", "Grenada", "Kiribati", "Liechtenstein", "Marshall Islands", "Monaco", "Nauru", "Palau",
        "Palestine", "Puerto Rico", "Saint Kitts and Nevis", "Saint Lucia", "Saint Vincent and the Grenadines",
        "Samoa", "San Marino", "Sao Tom and Principe", "Solomon Islands", "Timor-Leste", "Tonga", "Tuvalu",
        "Unknown", "Vanuatu", "Vatican City", "Western Sahara"
    ]
}

VALID_OPERATORS = [
    "Army", "Navy", "Air Force", "Defense Wide",
    "Ukraine (Assistance)", "Foreign Assistance", "Other"
]

SUPPLIER_LIST = [
    "22nd Century Tech", "A&P Group", "A&R Pacific -Garney Federal", "AAR Supply Chain Inc.", "Aardvark Clear", "AASKI Technology", "AAVCO", "Abacus Tech Corp", "Abdallah Al-Faris", "Abeking Rasmuss",
    "ABG Shipyards", "ABM Shipyard", "Absher Construction Co.", "Abu Dhabi MAR", "Abu Dhabi SB", "ACC Construction Co.", "Accenture", "Accurate Energetic Systems", "ACE Technology", "Aceinfo Solutions",
    "ACHILE Consortium", "Achleitner", "ACMI", "ACT-Corp", "ActioNet", "ADCOM Systems", "ADI Group", "Admiralty Ship", "Advanced Navigation and Positioning Corp.", "Advanced Technology International",
    "AdvElect Co (AEC)", "AECOM", "Aegis Technologies", "Aeraccess", "Aero Def Systems", "Aero Synergie", "Aero Vodochody", "Aerodata AG", "Aerodyca", "Aerojet Rocketdyne", "Aeromaritime Grp", "Aeromot",
    "Aeronautical Development Establishment", "Aeronautics Defense Systems", "Aerospace Corp", "Aerostar", "Aerostar S.A.", "Aerotree", "AeroVironment", "AeroVolga", "Affigent",
    "Africa Automotive Distribution Service", "Agat", "AgEagle", "Agiliti Health", "AICI-Archirodon JV", "AIDC", "AIM Defence", "Air Center Helicopters", "Air Tractor", "Airbus", "Airbus-Rheinmetall",
    "Airborne Tactical Advantage Co.", "Aircell", "Aircraft Readiness Alliance", "AirRobot", "AIS Engineering", "Akkodis", "Albadeey", "Albatross Industria Aeronautica Ltd.", "Alcatel-Lucent", "Alcock Ashdown",
    "Alexandria Ship", "Alion Science", "Allen-Vanguard", "Alliant Techsystems Operations", "Allison Transmission", "Alpha Marine", "Alpine Armoring Inc.", "Altawest", "Altec Industries", "ALTECH Services",
    "Aleut Federal", "Alzchem Trostberg", "AM General", "Amazon", "Amentum Services", "American International Contractors", "American States Utilities Services", "American SysCorp", "American Systems Corp",
    "AMESYS", "AMI Industries", "AMO ZIL", "Amper Group", "AMSL Aero", "AMTEC Corp.", "AMX International", "Amyx Inc", "AMZ-Kutno", "Anadolu Shipyard", "Analytic Services", "Ananda Shipyard", "Andrea Systems",
    "Andritz Hydro Corp.", "Anduril Industries", "Antonov", "ANVL", "AOI", "Apogee Engeineering", "Applied Mechanics", "Applied Technology", "Applied Visual Technology", "APS", "Aquacopters", "Aquila Aerospace",
    "Arab Contractors", "Arcfield Canada", "Archer Aviation", "Archer Western", "Arcturus UAV", "ARES Shipyard", "Aresa Shipyard", "ARGE K-130", "ARGE NNbS Consortium", "Argon ST Inc.", "ARGE DiNa 155", "ARIS",
    "Arma", "Armenian Air Force Institute", "American Electronics Warfare Associates", "American Ordnance", "Armour International", "Armoured Car Sys", "ARMSCOR", "Armtec Defense Products",
    "Arnold Defense and Elec", "ARO SA", "Arotech Corp", "Arquus", "Array Information", "Arrow Edge LLC", "AR-SAT", "Arsenal d' Marinha", "Arsenal JSCO", "ARTEC", "ARTEL, Inc", "ASC Pty Ltd",
    "Ascent Flight Training Consotium", "Ascom Group", "Aselsan", "ASENAV", "Asian ArmoredVeh", "ASIMAR", "ASISGUARD", "Ashot Ashkelon", "ASL Group", "Aslemetals Oy", "ASMAR", "ASRC Federal", "ASRY",
    "Assurance Tech", "Assured Information Security Inc.", "Aster Engineering", "Astilleros Armon Vigo SA", "Astilleros Navales", "ASTIMAR", "Astronics Test Systems", "ASTRUM", "AT&T", "A-techSYN",
    "Atheeb Integraph Saudi Co.", "Atlas", "Atlas Elektronik", "Atlas Group", "ATR", "August Schell Enterprises", "Aurora Flight Sciences", "Austal Limited", "Australian Target Systems", "Autoespar SA",
    "Automotive Ind Ltd", "AUVERLAND", "Aviation Repair Technologies", "Aviation Systems Engineering", "Aviation Training Consulting", "Avibras", "AVIC", "AVNL", "Avtech Corporation", "Avtokraz Holding Co",
    "AWEIL", "AWSR Shipping", "B&F", "Babcock Group", "BAE Systems", "Ball Corporation", "Baltic Workboats", "BAMS", "Bangkok Dock", "Barrett Comm", "Basler", "Bason Shipyard", "Bath Iron Works", "Battelle",
    "Baud Telecom Co", "Baykar", "Bechtel Group", "Becker Avionics", "Beechcraft", "Beherman Demoen", "Beijing JeepCorp", "Bell", "Bell Boeing", "Bell Textron", "Bellanca", "BEML-India", "Bender Shipbldg",
    "Bergen Group", "Beriev", "BGI-ASI JV", "Bharat Dynamics", "Bharat Elec Ltd", "Bharat Heavy Electricals", "Bharat Sanchar", "Bharati Shipyard", "Bigelow Family Holdings", "Bin Jabr Group", "Bird Aerosystems",
    "Birdon", "Bittium", "BL Halbert International", "Black Box Corp", "Black Micro Corp", "Black River Systems", "Blackberry", "Blackned", "BlackSky", "BlindermanPower", "Blue Air Training", "Blue Ivy Partners",
    "Blue River Consortium", "Blue Tech Inc", "Bluebird", "BlueHalo", "Boeing", "Boelwerf Shipyard", "Bollinger Shipyard", "Bombardier", "Booz Allen Hamilton", "Boresight", "Boustead DCNS JV",
    "Boustead Holding", "Bowhead", "Brahmos Ltd", "BrainGu LLC", "Bridgestone Aircraft Tire Inc.", "Britten-Norman", "Brodosplit Shipyard", "Brooke Marine", "Bryan 77 Construction", "BSVT", "BSVT-NT",
    "BT Group", "BUAA", "BwFuhrparkService", "BWI", "By Light", "CACI", "CAE USA", "Cairns Slipways", "Calian", "Calidus", "Cambridge Intl Systems", "Cammell Laird SB", "Canadair", "Cantieri Navali",
    "Carahsoft Inc.", "Cardama", "Carnegie Mellon University", "CASC", "CASIC", "C-Astral Aerospace", "C-AT", "Caterpillar", "CATIC", "CDO Technologies", "CDW Corporation", "CEA Tech Pty Ltd", "Celier Aviation",
    "CENTECH GROUP", "Cerbair", "Cessna", "CFM International", "CGI", "Chaiseri Metal & Rubber", "Changhe", "Chantier Davie Ship", "Charles Stark Draper", "Chas Kurz", "Chemring", "Chengdu",
    "Chowgule and Company", "Chrysler Group", "Chugach Technical Solutions", "Chung Shan Inst", "Cianbro", "CINAR", "CIO", "Cirrus Aircraft", "Cisco Systems", "Clark Construction Group",
    "CM de N (France)", "CNF Technologies", "CNIM", "CNN Navigation", "Coastal Defense Inc.", "Cobham", "Cochin Shipyard", "CODALTEC", "Codan", "Cohort plc", "Cohu Inc", "Colby Co. LLC",
    "Cole Engineering Services", "Collins Aerospace", "COLSA", "Colt's Manufacturing Co.", "Columbia Helicopters Inc.", "COM DEV International", "Comlenia", "Commander Aircraft Corporation",
    "Commtact", "Computacentre", "CompQsoft", "Computer World Services", "COMSOFT", "ComtechTelecomm", "Conco Inc.", "Conlog Group", "Conoship Intl", "Conquest USA", "Consigli Construction",
    "Consortium Management Group", "Conti Federal Services", "Continental Maritime", "Core Tech International", "Core4ce", "Corporacion De La Industria Aeronautica Colombiana", "Corvid Technologies",
    "COTECMAR", "CounterTrade", "CoVant Technologies", "CPMIEC", "Credence Mgmnt Sol", "Creotech", "Crew Training International", "CRIST", "Criterion Solutions", "CRL Technologies", "Crowley Maritime",
    "CRSA", "Crystal", "CSBC Corp., Taiwan", "CSC", "Cubic Corporation", "Cukurova Holding", "Cummins Inc.", "Curt Nyberg", "Curtiss-Wright", "Cybaero AB", "Dae Sun Shipbldg", "Daewoo", "Daher", "Daimler AG",
    "Dakota Creek", "Dalnyaya Radio", "Damen Shipyards", "DAMEX Shipbldg", "Danbury Mission Technologies", "Danish Maritime", "Danish Yacht", "Danyard Aalborg", "Darkhive", "DARPA", "Dassault", "Dassault Dornier",
    "Data Link Solutions", "Data Sys Analysts", "Datamir", "DataPath", "Day & Zimmerman Lone Star", "Dayton T. Brown Inc.", "DCCA", "DCD-DORBYL", "DCI", "DCNS Odebrecht", "DCS Corp.", "De Havilland Canada",
    "Dearsan Shipyard", "Decisive Analytics", "Deep Trekker", "Defense Ind Org", "Defense Industries Organization Of Iran", "Defense Solutions", "Defense Technology Institute", "Defenture", "Deftools",
    "Delaware Nation Industries Emerging Technologies", "Dell Inc", "Deloitte", "Denel", "Derecktor Shipyard", "DESA", "Design West Technologies", "Destini Berhad", "Detyens Shipyard", "DEW Ltd", "DFDS Group",
    "Diamond Aircraft", "DIANCA", "DIDEP", "Diehl", "Digital Angel Corp", "Digital Management", "Diligent Consulting", "Divelink Underwater", "Diversified Tech Svcs", "DJI", "DKW Communications", "DOF ASA",
    "Domo Tactical Communications", "Doosan Group", "Dornier", "Draken", "Draper Labs", "DRB-HICOM", "DRDC Canada", "DRDO", "Drew Marine USA", "DRS Network and Imaging Systems", "DSD Laboratories", "DSG",
    "DSN Corp", "DSTA", "Ducommun Inc.", "DXC Technology", "Dynamic Systems", "Dynamics Resrch", "Dynamit Nobel", "DynCorp Int'l", "Dynetics Technical Solutions", "Eastern Shipbuilding", "ECAN", "ECRN",
    "ECS Federal", "Emcube Inc", "Edgar Brothers", "Edge Autonomy", "EDGE Group", "Edison Chouest", "EFR Ltd", "eGlobalTech", "EID S.A.", "Eire Forge and Steel", "EINSA", "ELAC Sonar GmbH", "Elbit Systems",
    "ELBO", "Elebra", "Electra", "Electric Boat Corp.", "Electro Optic Systems", "Elettronica SpA", "ELINC", "EllisDon", "EM Solutions", "Embraer", "EMESEC", "EMGEPRON", "Emit Aviation", "EMPL Austria", "EMPORDEF",
    "EMS Tech", "EMT", "EMW", "ENAER", "EnerSys Energy Products", "Engility Corporation", "Engine Eng Oman", "ENICS", "Ensign Bickford", "Enstrom Helicopter", "Entrol", "Environics Oy",
    "Environmental Chemical Corp", "Envision Technology", "Envisioneering Inc.", "EONIC", "EOS", "EPC2 Consortium", "EPE", "EPIIC Consortium", "Eprius", "EPS Corporation", "Epsilon Systems", "ESSI/SEI",
    "Esterline", "Eurofighter", "EuroMIDS", "EUROPAAMS", "Eurosam", "Euroshop SA", "EuroSpike", "Exail", "Excellus Solutions", "Exeter Group", "Extra", "FABREQUIPA", "Fabryka Broni", "FAdeA",
    "Fairchild", "Famae", "FAME SAC", "Fasharkan Ship", "Fassmer", "FAW Group", "FCN Technology Sol", "FEDITC", "Federal Contracting", "FedStore Corp", "FemmeCompInc", "Fenix Air Inc.", "FFA", "FFA Emmen",
    "Firestorm Labs", "Five Rivers Analytics", "Fiat Group", "Fiat-Leonardo", "Flatter Inc.", "Fidelity Technologies", "Fincantieri", "Fischer Panda", "Flensburger Fahrzeugbau", "Flensburger SB",
    "Flight Technologies", "Flightcell Intl", "FLightSafety", "FLIR", "Fluor Marine Propulsion", "Flyer", "Flying Legend", "FMA", "FN Herstal", "FNSS", "Fokker", "Force 3", "Ford Motor Co.",
    "Forum Energy Technologies", "FREIRE Shipyard", "Frequentis GmbH", "Fresia SPA", "Frontgrade Technologies", "Frontier Electronic Systems", "FSC Lublin Auto", "FSUE Neptune", "Fujitsu",
    "Furuno Electric", "G & F Technology", "G1 Aviation", "Gabler Maschine", "GAF", "Game Composites", "Garco Construction Inc.", "Garden Reach SB", "Garmin", "Gate Elektronik", "GC Rieber Shipping",
    "GDELS-Mowag", "GECI", "Gemelli", "Genasys", "GenCorp", "General Atomics Aeronautical Systems", "General Dynamics", "General Electric", "General Motors", "Generic Supplier", "Geneset Powerplants",
    "Georgia Tech", "German Naval Yards", "GESPI", "GFE", "Gibbs & Cox Inc.", "GIDS", "Gilbane Federal", "GKN Aerospace", "Gladding-Hearn", "Global GndSpt LLC", "Global Military Products", "Global Services LLC",
    "Global Tech Res", "Global Technical Sys", "Globecomm", "GMV Aerospace and Defense", "Goa Shipyard Ltd", "Golcuk", "Goodrich Corp.", "Goodyear Tire and Rubber Co.", "Grabba", "Granite-Obayashi",
    "Granta Autonomy", "Grevicom SAC", "Griffon Corporation", "Grob", "GRYFIA", "GTRI", "Guimbal", "Guizhou", "Gulf Island Marine Fabricators LLC", "Gulfstream", "Guyco Inc.", "GZAS", "H2O Guam JV", "Hadean", "Hai Minh Corporation", "Haivision Systems Inc.", "HAL", "Hanjin Indust'l SB", "Hanwha", "Harbin", "Harland & Wolff", "Harper Construction", "Harris", "Harwar International Aviation Technology", "Hatehof", "Hawaiian Rock Products Corp.", "HB Utveckling AB", "HAVELSAN", "HDT Expeditionary Systems", "Head/Diaz 2022", "Heavy Ind. Taxila", "Heckler & Koch", "Helibras", "Helicentro Peru SAC", "Hellenic Aerospace Industries", "Hellfire LLC", "Hensel Phelps Construction", "Hensoldt", "HESA", "HexagonComposite", "Hinduja Group", "Hindustan Ship", "Hi-Q Engineering", "Hisdesat SA", "Hitachi", "Hitachi Kokusai", "Hitzler Werft", "HKV", "Hodges Transportation", "Honeywell", "Hong Ha Shipbuilding", "Hong Leong Group", "Hongdu", "Horizon Technologies", "Hornbeck Offshore Operators", "Howe and Howe", "HP", "HPI Solutions", "HTX Labs", "Hughes Comm", "Humbert Aviation", "Huneed Tech", "Huntington Ingalls", "Huta Stalowa Wola", "HV Joint Venture", "Hydra Technologies", "Hydrema", "Hyundai", "Hyundai J Comm", "IAI", "IAP Worldwide Svc", "IAR", "IBM", "ICF", "Icom Inc.", "ICOMM Tele Ltd.", "IdeaForge", "iGOV", "IHI", "II-VI Aerospace and Defense", "Ilyushin", "ImagineOneT&M", "IMBEL", "IMC Group", "IMCO", "immixGroup", "IMMSI SPA", "IMPSA", "Imtech Marine", "INACE", "Indonesian Aerospace", "Indra", "Indrasoft", "INDUS Technology", "InDyne", "InfoReliance Corp", "Infotron", "Inmarsat", "Innocon", "Innnovaero", "Insitu", "Insta ILS", "Institute of International Education", "INTA", "Integ Surv Tech", "Integral Consulting Services", "Integral Systems", "Integrated Convoy", "Integrated Defense Solutions/Greit", "Integrated Dynamic", "Integrated Dynamics", "Integrated Surveillance and Defense", "Integration Innovation", "Intelligent Decisions", "Intelligent Waves", "INTELSAT", "Inter-Coastal Electronics", "InterCaribbean Airways", "Intermarine", "International Business Machines Corp.", "Intl Shipholding Corp", "Intman SA", "Intracom SA", "INVAP", "Invariant Corp.", "INVISIO", "IOMAX", "IPS Inc", "Iridium Satellite", "Iron Bow Tech", "Irving Shipbldg", "Israel Military Industries", "Israel Shipyards", "ISRO Internal", "Istanbul Shipyard", "Isuzu Motor Co", "Italcantieri", "Italtel", "Italthai Marine", "ITG", "ITI Limited", "ITP Aero", "ITT", "Iveco Defence Vehicles", "Iveco-Oto Melara Consortium", "IVEMA", "IWI", "IXBlue", "Izhmash Unmanned Systems", "Jacobs Eng Group", "Jacobs/B&V JV", "James Fisher", "Jankel", "Japan Marine United", "Japan Steel Works", "Javelin JV Team", "JCB", "Jelcz-Komponenty", "Jet Tekno", "JetZero", "JF Taylor", "JHU/APL", "Joby Aviation", "Johns Hopkins University", "Jong Shyn Ship", "JRC Group", "JSC Almaz-Antey", "JSC Kurganmashzavod", "JSC Tactical Missiles Corp", "Junghans Microtec", "Jupiter Wagons Ltd.", "KADDB", "Kader", "KAI", "Kaman", "KAMAZ", "Kamov", "Kangnam Corp", "Karachi Shipyard (KSEW)", "Katmai Management Services", "Katmerciler", "KATO Engineering", "Kawasaki", "Kay and Associates", "Kazakhstan Eng", "Kazan", "KBM Kolumna", "KBP Instrument", "KBR", "Kearfott Corp", "Keppel Corp", "Kerametal", "Kership", "Khan ResLabs", "Kharkiv Morozov", "Khulna Shipyard", "Kiewit-Alberici SIOP MACC", "King ICT", "King Technologies", "KIRINTEC", "KNDS", "Knight Sky", "Knights Armament Co.", "Koam Engineering", "Koc Group", "KomatsuIndustries", "Kongsberg", "KONSTRUKTA", "Kord Technologies", "Korea Shipbuilding & Offshore Engineering", "Korte Construction", "Agency for Defense Development", "Korean Air Aerospace Division", "KRAS - India", "Krasmashzavod", "Kratos Defense", "Kronshtadt Group", "Krauss-Maffei Wegmann", "Kryukov Car Bldg", "KT Consulting", "KVH Industries", "Kyndryl Finland", "L3 Technologies", "Lancair", "Landmarc", "Lane Construction Corp.", "Larsen & Toubro", "Leidos", "Leonardo", "LET", "Level 3 Comm", "Life Cycle Engineering", "LG Group", "LIG Nex1 Co", "LinQuest Corp", "LinTech Pragmatics JV", "Lite Comms LLC", "Lockheed Martin", "Loc Performance Products", "LOM PRAHA", "Long Wave Inc", "Longbow LLC", "Loral", "Lumen", "Lumenier", "Lung Teh Shipbldg", "Lurssen Group", "Lutch", "Lutsk", "M Ship Co", "M1 Support Services", "M2 Technologies", "M7 Aerospace", "Mach Industry Grp", "Mack Defense", "Mackay Comm", "MAESTRAL", "Maestranza AMSU", "MAG Aerospace", "Magellan Aerospace Corporation", "Mahindra", "MA Mortenson", "MAN", "Manhattan Construction", "ManTech", "Mapiex Aviation", "Marine Alutech Oy", "Marine Hydraulics", "Marine United", "MarineTec", "Marinette Marine Corp.", "MARS Shipyards", "Marsh Aviation", "Marshall Aerospace", "MARSS", "Marsun Company", "Martifer Group", "Marvin Land System", "Mastodon Design", "Mathtech", "Maule Air", "MAV", "Maxar Technologies", "Mazagon Dock", "MBB", "MBDA", "McCrone Associates", "McDermott Marine", "McLean Contracting", "MD Helicopters", "MDA Space", "MDT Armour", "MechDB S Africa", "Mectron", "MEDAV GmbH", "Mercedes-Benz", "Mercer Engineering Research Centre", "Mercury Systems Inc.", "Merlin Labs Inc.", "Merwede", "Mesko", "MESIT holding", "Messer Construction", "Metal Shark", "MetalCraft Marine", "Metalnor SA", "Meyer Werft", "Michelin", "Micro Aviation", "Microdis Electronics", "Micropol Fiberoptic AB", "Microsoft", "MicroTech", "Middle East Def", "Mikal Group", "Mikoyan", "Mil", "MilDef", "Milenium Veladi Corp.", "Millenium Space", "MilSOFT Software", "MineWolf Systems", "MISC Berhad", "Mission1st", "Mistral Inc.", "Mitie", "Mitsubishi", "Mitsui SB", "MKEK", "MMIST", "MNDI Pacific JV", "MO Porte-Avions", "Modern Technology Solutions", "Moller-Maersk", "Moog Inc.", "MorseCorp Inc.", "Morye Shipyard", "Motorola Solutions", "MSI", "Mudry", "Mugin", "MVL USA", "MW Builders", "Mythics", "Nakilat", "Nakupuna Consulting", "NAMC", "Nammo", "Nan Inc.", "Nanchang", "National Academy of Sciences of Belarus", "National Steel and Shipbuilding", "Natl Radio Telecom", "Nautica Nova", "Naval Gijon Ship", "Naval Group", "Naval Shipyard Gdynia", "Navantia", "Naviris", "Navistar International", "Navmar", "NCI Info Sys", "NCSIST", "ND Defense", "NDMA", "NEC", "Neiva", "Neorion Group", "NES Associates", "NetCentrics Corp", "Netline Comm", "New Directions Technologies", "NewSpace India", "NEWTEC", "Nexter", "NGV Tech", "NH Industries", "NICCO Comm", "Nigerian Dockyard", "NII STT", "Niigata Shipbuilding", "NIMR Auto", "Nissan", "nLIGHT Nutronics", "Noble Supply and Logistics", "Noblis MSD", "Nokia", "Nordic Terrain Solutions", "Norinco", "Norma Precision AB", "Nortel", "North Sea Boats", "Northrop Grumman", "Northstar Aviation", "Nostromo", "Novadem", "NP Aerospace", "NPO Elektro'ka", "NPO Lavochkin", "NRL", "NSSL", "NSWC", "NT Service", "NTConcepts", "nTSI", "NTT Group", "NUBURU", "Nurol Co.", "NVL Group", "Oakwell Engineer", "OBRUM", "OCEA Group", "Ocean Shipholdings", "Ocean Tech Sys", "Oceaneering", "OCR Global", "Odebrecht Group", "Odyssey Systems Consulting Group", "OGMA", "OHB System AG", "OIP Land Systems", "Old North Utility Services", "Olin Winchester", "Omnisec AG", "Omnisys", "Ondas", "Optics1 Inc.", "Optima Government Solutions", "Orbit Technologies", "Orizzonte Sistemi Navali", "Orskov Group", "Oshkosh", "OSI", "Otobus Karoseri", "Otokar", "OTT Technologies", "Out of Business", "Overaasen AS", "Ovzon", "PAC", "Paccar", "Pacific Aerospace", "Pacific Rim Constructors", "Pacifics Propeller International", "PAE Aviation and Technical Services", "Pakistan Aeronautical Complex", "Palantir Technologies", "Palantir USG", "Palfinger", "PAMA-SP", "PanAmSat", "Panavia", "Panha", "Paramount Group", "Parker-Hannifin", "Parrot", "Parsons Government Services", "Patria", "Patriot Contract Svcs", "PCCI", "PCM", "Pearson Engineering", "Peerless Technologies", "Pelatron", "Pelegrin", "Penman Company", "Peraton Technology Services", "Persistent Systems", "Peterson Bldrs", "PGSUS", "PGZ", "PGZ-PILICA Consortium", "PGZ-NAREW Consortium", "Phacil Inc", "Phoenix Air Group", "Philadelphia Yard", "Philippine Telephone", "Piaggio", "Pilatus", "Pindad", "Piper", "Pipistrel", "Piriou Naval Svcs", "PJ Aviation", "PKL Services", "Plath", "PN Dockyard", "Polaris Industries", "Polish Defence Holding", "Polska Grupa", "Polskie Zaklady Lotnicze", "Poly Technologies", "Polyot", "Polysentry", "Pragmatics", "Presidio", "Priboy", "PRIMA Research", "Proforce Defence", "Programs Management Analytics and Technologies", "Propmech Corp", "Prox Dynamics", "PS Engineering", "PSI", "PSM", "PT Batam", "Pt Bhinneka Dwi Persada", "PT Citra Barahi Shipyard", "PT Daya Radar Utama", "PT Dirgantara", "PT Dumas Shipyard", "PT Kodja Bahari", "PT PAL Indonesia", "PT Palindo", "PT Republik Defensindo", "PZL-Mielec", "PZL-Okęcie", "PZL-Swidnik", "Q-Techn LLC", "Qbase, LLC.", "QED Systems Inc", "QinetiQ", "Qioptiq", "Qods Aviation Industries", "Quad City Aircraft", "QualX Corp.", "Quantum Research", "Quantum Systems", "QuantX Labs", "Qwest", "R&W Contractors", "Radiance Tech", "Radmor SA", "Rafael", "RAM Systems", "RAMET", "Range Generation Next", "Rannoch Corp", "Rauma Marine", "Ravenswood Solutions Inc.", "RAVN Group", "Raytheon Technologies", "RC Construction", "Rebellion Defense", "ReconCraft LLC", "Record Steel & Construction", "Red Peak Technical Services", "Red River Computer", "Redflex Group", "Redwire", "Regional One", "Reims-Cessna", "Reiser", "Reliance Defence", "Reliance Test and Technology", "Remdiesel", "Remontowa Group", "Remoy Shipping", "Renk America", "Repkon USA-Defense", "Reshetnev Company", "Ressenig", "Reunert", "Revolution Aviation", "Rh-Alan", "RHEA Group", "Rheinmetall", "Rheinmetall BAE Systems Land", "Rheinmetall Denel", "Rheinmetall MAN", "Ribcraft USA", "Ricardo PLC", "Riga Shipyard", "RIO Design Bureau", "Rio Santiago Shipyard", "Rise8 Inc.", "RiverHawk Group", "Robertson Fuel Systems", "Robin Radar Systems", "Robinson", "Roboteam", "Rocket Lab National Security LLC", "Rockwell Collins", "Rodman Group", "Rohde & Schwarz", "Roke", "Roketsan", "Rolls-Royce plc", "Roman Brasov", "ROMARM", "Rosomak", "Rostec", "Rothe Development", "Rovsing A/S", "RQ Construction", "RS-UAS", "RTX", "RUAG", "RV Connex", "RWG Repair & Overhauls USA", "Saab", "Sabiex Group", "Sabre Systems", "Sabreliner Corporation", "SAFAT", "Safe Boats Intl", "Safran", "Sagemcom", "SAIC", "Sako", "SAL", "Salient CRGT Inc", "Sallyport Global Holdings", "SAN", "San Yang", "Sandia Nat Labs", "Sanmina-SCI", "Sanska", "Santana Motors", "Santierul Naval", "SANUKI Shipbldg", "SAPURA", "Sapura Thales", "Sarco Defense", "Sasebo Heavy Ind", "Satuma", "Savox Communications", "SBIC", "Scandinavian Avionics", "Scania", "Scheepvaart KB", "Schiebel", "Schweizer", "Schutt Industries", "Science and Engineering Services", "Science Applications International", "Scientia Global", "Scientific Research Corp.", "Scorpene JV", "SCOTTY Group", "SCR", "SEA", "Seabird Aviation", "Sealift Inc", "Seaspan Marine", "Seaward Marine Services", "Second-Hand", "Sectra Comm Sys", "SecuriGence", "Sedef Shipbuilding", "Seed Innovations", "Seemann Composites LLC", "Sefine Shipyard", "Segue Technologies", "Selah Shipbuilding", "SELEX Elsag", "Selex ES", "SEMAN Peru", "SEPECAT", "SEPI", "Sepura", "Serbian State", "Serco Group plc", "SES", "SETEL/REMSCO", "SGJV", "Shaanxi", "Shaanxi Auto Grp", "Shenyang", "Shijiazhuang", "Shin Maywa Industries", "Shin Yang", "Shoft Shipyard", "Short Brothers", "SI Systems Technologies", "SICC", "Sielman S.A.", "Siemens", "Sierra Nevada", "SIG Sauer", "Sigen Consortium", "Sikorsky", "Silent Sentinel", "Silver Ships Inc.", "SIMA Peru", "Singapore Tech.", "SingTel Group", "SISDEF", "Sistemprom", "Sisu Auto", "SITAB Consortium", "SK Holdings", "Skanska", "SkyAlyne", "Skydio", "Slingsby", "SmartShooter", "Smartronix Inc", "SMS Data Products", "SNC-Lavalin", "SNVI", "Sobeca", "Soby Vaerft", "Socarenam", "Socata", "Sodexo Management Inc.", "SOFIS-TRG", "SOFRAME", "Sojitz Corporation", "Soko", "Solar Industries", "Solers", "Solstad Offshore", "SONAK", "Sonalysts Inc.", "Songthu Corporation", "Southern African Ship", "Southern Maryland Electric Cooperative", "Southern Resc'h", "Southwest Resc'h", "Soviet Tank Plant", "Sozvezdie JSC", "SPA", "Spaceflux", "SpaceX", "Spanish Missile Systems", "Sparton De Leon Springs LLC", "Special Technology Ctr", "Spectra", "Spectrum Comm", "SpearUAV", "SpeedCast", "Sprint", "SR Telecom", "SRC", "SRCTec", "ST Aerospace", "StandardAero Inc.", "Stark Aerospace", "Stauder Technologies", "Sterling Computers", "Steyr", "STG", "Stinger ProjectGP", "STM Group", "Streit Group", "STS International", "STX Corporation", "Subaru", "Submarine Manufacturing and Products", "Sukhoi", "Sumaria Systems", "Sumidagawa Ship", "Sumitomo", "Summit Aviation", "Sunair", "Supacat", "Superior Govt Sol", "Superior Marine Ways", "Surrey Satellite Technology", "Survey Copter", "Suzuki Motor Corp", "SVI Engineering", "Swan Hunter", "Swecon", "Swede Ship", "SwedishSpace Cp", "Swiftships SB LLC", "Symetrics", "Synectic Group", "Sypaq Systems", "Sypris Solutions", "Sys for Def/GVS", "System Studies & Simulation", "Systematic", "Systems Planning and Analysis", "T. Mariotti", "Tactical Air Support Inc.", "Tactical Engineer", "TADANO", "TAE Aerospace", "TAI", "Talbert Manufacturing Inc.", "Target Technologia", "Taskizak Shipyard", "TAT Technologies", "Tata Advanced Systems", "Tata Group", "TATRA", "Taurus Systems", "Taylor Defense Products", "TCG", "TCIL", "TDW GmbH", "TDX International", "Technica Corp", "Technical Comms", "Technology Unlim", "Tecnam", "TECNOBIT", "Tekever", "Telecomm Systems", "Teledyne", "Teledyne FLIR", "Telephonics Corp.", "Telespazio", "Teletronics Technology", "Telia Finland", "Tellumat", "Telos Corp", "Telstra", "Terberg Group", "Terma A/S", "Tesat Spacecom", "TESCO INDOMARITIM", "TESLA", "TESS Defence", "Tesseract Ventures", "TETRAEDR", "Texas A&M", "Textron", "Thales", "Thales Alenia Sp", "ThalesRaytheon", "The MIL Corp.", "The Whiting-Turner Contracting Co.", "TWPG", "THEON International", "ThirdEye", "Thoma-Sea Ship", "Thrane & Thrane", "Threod Systems", "Thuraya", "ThyssenKrupp AG", "Timken Gears & Services", "Titan Aircraft", "TKC Global Solutions", "TNO", "Tobyhanna Army", "Tomahawk Robotics", "Top Aces", "Toshiba", "Toyota Motor Corp", "Trans-Ce Cargo SA", "Transall", "Transas Group", "Transbit", "Transfield Services", "TRAX International", "TrellisWare Tech", "Trideum Corp", "Triman Industries", "Triton Group Hold", "TRU Simulation Plus Training", "TRX System", "TSS Solutions", "TTC TELEKOM", "TUBITAK", "Tupolev", "Turkish AFF", "Turner Construction", "Twin Commander Aircraft", "TYBRIN", "Tyco Intl", "Tyovene", "Tyto Athene", "Tyvak International", "UAV Communications", "UAV Solutions", "Uavision Aeronautics", "UCOCAR", "Uconsystem", "UK Docks Marine Services", "Ukraine Weapons", "UkraineTank Plant", "Ukroboronprom", "Ukrspecsystems", "Ulijanovsk", "UltiSat", "Ultra Dimensions Pvt. Ltd.", "Ultra Electronics", "Ultra I&C", "Ultra Maritime", "UMM", "Umoe Group", "Unicom", "Unicom Government", "Unknown", "Unified Industries", "UNIMO Technology", "Unimor Radiocom", "Unisys", "Unit Co.", "United Crane and Excavation Inc.", "United Electronics", "United Launch Alliance", "Univ of Texas", "Univ of Toronto", "Universal Shipbldg", "Unman'dDynamics", "Ural Works Civil Aviation", "Uralvagonzavod", "URC Systems", "UROVESA", "US Marine Inc", "US Ordnance", "USCG YARD", "UTVA", "UVision Global Aero", "Valero Marketing and Supply", "Valiant Global Defense", "Valkyrie Aero", "Van's Aircraft", "Multiple", "Vector Scientific", "Vector Solutions", "Vectrus Systems Corp.", "Vega Company", "Vencore", "Veritas Capital", "Verizon", "Vertex Aerospace", "Vertex Standard", "Vestel", "ViaSat Inc", "Victory Solutions Inc.", "VideoRay LLC", "Viettel Group", "Vigor Industrial", "Viking Air", "Viking Arms", "Vimpel", "Vladimir Radio", "Volkswagen Group", "Volvo Group", "Von Wolf", "VOP 025", "VOP 026 Sternberk", "VPK", "VSE Corp.", "Vulcanair", "V2X", "Walsh Federal LLC", "Wartsila", "Watterson Construction Co.", "WB Electronics", "WBA Blindajes Alemanes", "West Sea Shipyard", "Weststar Group", "WG Yates and Sons", "Wildflower Intl", "Windmill Intl", "World Wide Tech", "WULCO Inc.", "WZE", "WZM", "X-Bow", "Xian", "Xian ASN Technical Group", "XTAR", "Yakovlev", "Yamaha", "Yaroslavl Radio", "Yeonhab Precision", "Yokohama Yacht", "Yoland Corp.", "Yonca-Onuk", "Yugoimport-SDPR", "Zala", "Zamil Offshore", "Zen Technologies", "Zenair LTD", "Zenit Shipyard", "Zenith", "Zlin", "Zwijnenburg", "ZyXEL Comm", "Hydroid Inc", "West Coast JV,", "University of Dayton Research Institute", "Saguaro Business Solutions LLC", "Learjet", "General Dynamics Electric Boat", "Ball Aerospace & Technologies", "TCOM", "Raytheon Missiles and Defense", "Lockheed Martin Missile and Fire Control", "EFW", "Amherst Systems", "Lockheed Martin Sippican", "Hamilton Sundstrand", "Northrop Grumman Aerospace", "R.A. Burch Construction", "Lockheed Martin – Rotary and Mission Systems", "Trace Systems", "Northrop Grumman Space Systems Sector", "L-3 Communications Integrated Systems", "Flint Electric Membership", "Gray Analytics", "Lockheed Martin Aeronautics", "Lockheed Martin Space", "LTM Inc", "Alberici-Mortenson", "Atlantic Signal", "Haight Bey & Associates", "Container Research Corp", "Essex Electro Engineers", "TechFlow Mission Support", "Chugach Range and Facilities Services", "Raytheon Space and Airborne Systems", "Innovative Scientific Solutions", "Delavan", "Covalus", "Chromalloy Component Services", "Armorworks Enterprises", "Metro Machine", "Alloy Surfaces", "Valley Tech Systems", "Keysight Technologies", "Azure Summit Technology", "Isometrics", "Stratascorp", "Synergy Electric Company", "Custom Manufacturing & Engineering", "East West Industries", "MPR Associates", "ARCTOS Technology Solutions", "Enlighten IT Consulting", "Barrett Firearms", "Ametek Programmable Power", "Applied Physical Sciences", "SupplyCore", "Federal Resources", "General Atomics", "Penguin Computing", "Mancon", "Integrated Marine Services", "Compass Systems", "DRS Sustainment Systems", "IronMountain Solutions", "Ball Aerospace & Technologies", "Yulista Services", "SyQwest", "Advanced Technology Systems", "Cleveland Construction", "Canadian Commercial Corp", "Systima Technologies", "Ocean Ships", "Metro Machine Corp", "ImSAR LLC", "Systems Application & Technologies", "Twin Disc", "Konecranes Nuclear Equipment and Services", "Progeny Systems", "WEBCO", "REEL COH", "Waterman Transport", "Western Metal Supply", "Security Signals", "Wolverine Tube", "BC Customs LLC", "TLD America", "Crane Technologies Group", "IDSC Holdings", "AAR Manufacturing", "B & D Electric", "Vector CSP LLC", "Accurate Machine & Tool Corp", "Mississippi State University", "Stephenson Stellar Corp", "Earthly Dynamics", "Woolpert Inc", "Halter Marine", "Marion Manufacturing", "FN America LLC", "CDM Constructors", "Florida State University - Center for Advanced Power Systems", "International Marine & Industrial Applicators", "Zodiac-Poettker HBZ JV II LLC", "DigiFlight", "Globe Composite Solutions", "Meggitt Polymers and Composites", "Martin-Baker Aircraft", "United Kingdom Ministry of Defence", "Ultimate Training Munitions", "PAS Technologies", "DCM Clean Air Products", "Management Services", "Technology Service Corp", "General Electric Aviation", "ACME/RHB", "Howell Industries", "Airdyne Aerospace", "Dominion Energy", "Bionetics", "Choctaw Defense Manufacturing", "Centauri", "DRS Naval Power Systems", "Sentry View Systems", "ERAPSCO", "AAR Government Services", "Management Services", "L3 Doss Aviation", "AgustaWestland Philadelphia", "Marvin Engineering", "Collins Elbit Vision Systems", "FGS", "Navistar Defense LLC", "Voith Hydro", "Delfasco"
]

PROGRAM_TYPES = [
    "Training", 
    "Procurement", 
    "MRO/Support", 
    "RDT&E",
    "Upgrade", 
    "Other Service",
    "Unknown"
]

DOMESTIC_CONTENT_OPTIONS = [
    "Indigenous", 
    "Imported", 
    "Local Assembly", 
    "License Production"
]

GPT COde testing 

import json
import datetime
import time
import pandas as pd
import re
import difflib
import os
from dateutil import parser
from dateutil.relativedelta import relativedelta
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from openai import OpenAI

# Imports from other files
from config import MODEL_NAME, BASE_URL
from data.taxonomy import (
    TAXONOMY_STR, VALID_DEPENDENCIES, GEOGRAPHY_MAPPING,
    VALID_OPERATORS, SUPPLIER_LIST, PROGRAM_TYPES, DOMESTIC_CONTENT_OPTIONS
)
from src.prompts import (
    GEOGRAPHY_PROMPT, FINANCIAL_PROMPT, DOMESTIC_CONTENT_PROMPT
)

# ==========================================
# ✅ MEMORY FILE PATH (STREAMLIT CLOUD SAFE)
# ==========================================
MEMORY_FILE_NAME = "Market Segment.xlsx"   # <-- Must be saved in app root by Streamlit uploader

vectorizer = None
example_vectors = None
df_examples = None


def load_memory():
    """
    Loads Memory Excel and builds TF-IDF vectors.
    Works on Streamlit Cloud & local systems.
    """
    global vectorizer, example_vectors, df_examples

    vectorizer = None
    example_vectors = None
    df_examples = None

    try:
        if os.path.exists(MEMORY_FILE_NAME):
            df_examples = pd.read_excel(MEMORY_FILE_NAME)

            if "Description of Contract" not in df_examples.columns:
                print(f"⚠️ Memory load failed: 'Description of Contract' column not found in {MEMORY_FILE_NAME}")
                return

            # Build TF-IDF
            vectorizer = TfidfVectorizer(stop_words="english")
            example_vectors = vectorizer.fit_transform(df_examples["Description of Contract"].astype(str))

            print(f"✅ Memory loaded successfully from '{MEMORY_FILE_NAME}' | Rows: {len(df_examples)}")
        else:
            print(f"⚠️ Memory file not found: '{MEMORY_FILE_NAME}' (Memory disabled)")

    except Exception as e:
        print(f"❌ CRITICAL: Failed loading memory file '{MEMORY_FILE_NAME}': {e}")
        vectorizer, example_vectors, df_examples = None, None, None


# ✅ Load Memory at import time (Streamlit Cloud safe)
load_memory()


# ==========================================
# 1. HELPER FUNCTIONS
# ==========================================
SORTED_SUPPLIER_LIST = sorted(SUPPLIER_LIST, key=len, reverse=True)


def get_best_taxonomy_match(extracted_name: str) -> str:
    if not extracted_name or extracted_name.lower() in ["unknown", "not applicable", "multiple"]:
        return "Unknown"

    clean_name = extracted_name.strip()
    clean_name_lower = clean_name.lower()

    # Exact Match
    for supplier in SORTED_SUPPLIER_LIST:
        if clean_name_lower == supplier.lower():
            return supplier

    # Brand Name Filtering
    first_word = clean_name.split(" ")[0]
    candidates = [s for s in SORTED_SUPPLIER_LIST if first_word.lower() in s.lower()]

    if candidates:
        matches = difflib.get_close_matches(clean_name, candidates, n=1, cutoff=0.4)
        if matches:
            return matches[0]

    # Global Fuzzy Match
    matches = difflib.get_close_matches(clean_name, SORTED_SUPPLIER_LIST, n=1, cutoff=0.7)
    if matches:
        return matches[0]

    # Substring Safety Net
    for supplier in SORTED_SUPPLIER_LIST:
        if supplier.lower() in clean_name_lower:
            return supplier

    return clean_name


def call_llm(prompt_text: str, system_message: str = "You are a helpful assistant. Please respond in JSON format.") -> dict:
    """
    Safe LLM wrapper.
    If API fails, returns {} instead of crashing.
    """
    time.sleep(0.2)

    try:
        client = OpenAI(base_url=BASE_URL)

        response = client.chat.completions.create(
            model=MODEL_NAME,
            messages=[
                {"role": "system", "content": system_message},
                {"role": "user", "content": prompt_text}
            ],
            temperature=0,
            response_format={"type": "json_object"}
        )

        content = response.choices[0].message.content
        if not content:
            print("⚠️ LLM returned empty response content.")
            return {}

        return json.loads(content)

    except Exception as e:
        print(f"❌ LLM Call Error: {e}")
        return {}


def get_similar_example(new_text: str):
    """
    Finds the single most similar contract from the analyst memory using TF-IDF.
    Returns the text and correct classification.
    """
    if vectorizer is None or df_examples is None or example_vectors is None:
        return None

    try:
        new_vec = vectorizer.transform([str(new_text)])
        similarities = cosine_similarity(new_vec, example_vectors).flatten()
        best_idx = similarities.argmax()

        if similarities[best_idx] > 0.1:
            row = df_examples.iloc[best_idx]
            return {
                "text": row["Description of Contract"],
                "classification": {
                    "Market Segment": row.get("Market Segment", "Unknown"),
                    "System Type (General)": row.get("System Type (General)", "Unknown"),
                    "System Type (Specific)": row.get("System Type (Specific)", "Unknown"),
                    "System Name (General)": row.get("System Name (General)", "Unknown"),
                    "System Name (Specific)": row.get("System Name (Specific)", "Unknown"),
                    "System Piloting": row.get("System Piloting", "Derived from logic")
                }
            }

    except Exception as e:
        print(f"❌ Error finding similar example: {e}")

    return None


def calculate_derived_fields(financial_data: dict, geo_data: dict, description: str, contract_date_str: str) -> dict:
    try:
        start_date = pd.to_datetime(contract_date_str, dayfirst=True)
    except Exception:
        start_date = datetime.datetime.today()

    signing_month = start_date.strftime("%B")
    signing_year = str(start_date.year)

    val_llm_output = financial_data.get("Value (Million)", "0.000")
    try:
        clean_val = str(val_llm_output).replace(",", "").replace("$", "").replace("M", "").strip()
        val_float = float(clean_val)
        val_formatted = "{:.3f}".format(val_float)
    except (ValueError, TypeError):
        val_formatted = "0.000"

    cust_country = geo_data.get("Customer Country", "Unknown")
    supp_country = geo_data.get("Supplier Country", "Unknown")
    deal_type = "B2G" if (cust_country == "USA" and supp_country == "USA") else "G2G"

    program_type = financial_data.get("Program Type", "Other Service")
    mro_duration = "Not Applicable"

    if program_type == "MRO/Support":
        desc_date_str = financial_data.get("Description Date Found", "")
        if desc_date_str and desc_date_str.strip() != "":
            try:
                end_date = parser.parse(desc_date_str, fuzzy=True)
                diff = relativedelta(end_date, start_date)
                total_months = diff.years * 12 + diff.months
                mro_duration = str(max(0, int(total_months)))
            except Exception:
                try:
                    months_match = re.search(r"(\d+)\s*months?", desc_date_str, re.IGNORECASE)
                    years_match = re.search(r"(\d+)\s*years?", desc_date_str, re.IGNORECASE)
                    if months_match:
                        mro_duration = months_match.group(1)
                    elif years_match:
                        mro_duration = str(int(years_match.group(1)) * 12)
                except Exception:
                    mro_duration = "Unknown"

    qty = financial_data.get("Quantity", "Not Applicable")
    if program_type != "Procurement":
        qty = "Not Applicable"

    return {
        "Supplier Name": financial_data.get("Supplier Name", "Unknown"),
        "Program Type": program_type,
        "Expected MRO Contract Duration (Months)": mro_duration,
        "Quantity": qty,
        "Value Certainty": financial_data.get("Value Certainty", "Confirmed"),
        "Value (Million)": val_formatted,
        "Currency": "USD$",
        "Value (USD$ Million)": val_formatted,
        "G2G/B2G": deal_type,
        "Signing Month": signing_month,
        "Signing Year": signing_year
    }


# ==========================================
# 2. MAIN PROCESSOR
# ==========================================
def classify_record_with_memory(description: str, contract_date_str: str) -> dict:
    """
    Main entry point for processing a single row.
    Integrates:
    - TF-IDF Analyst Memory (Market Segment, Systems, Names, Piloting)
    - Geography, Domestic Content, Financials
    """

    # ✅ Reload memory each time to ensure latest upload works in Streamlit Cloud
    # (Streamlit reruns scripts often, but this ensures consistency)
    if vectorizer is None:
        load_memory()

    # --- A. MEMORY CLASSIFICATION ---
    similar_case = get_similar_example(description)

    system_instruction = f"""
    You are a Defense Contract Analyst.
    Your goal is to extract technical data points from the "Input Text".

    REFERENCE TAXONOMY:
    {TAXONOMY_STR}
    """

    user_message = f"Input Text: {description}\n\n"

    if similar_case:
        user_message += f"""
        IMPORTANT REFERENCE - Here is a similar contract classified by a human analyst.
        Use this as a guide for your logic:

        [Past Input]: {similar_case['text'][:300]}...
        [Past Correct Output]: {json.dumps(similar_case['classification'])}

        Now, apply the same logic to the current Input Text.
        """

    user_message += """
    --------------------------------------------------------
    REQUIREMENTS:
    1. Classify 'Market Segment', 'System Type (General)', 'System Type (Specific)' using the Taxonomy.
    2. Extract 'System Name (Specific)' (e.g., MC-130J) and 'System Name (General)' (e.g., C-130).
    3. Determine 'System Piloting' (Crewed, Uncrewed, or Not Applicable).
       - Software/Services/Ammo/Infra = "Not Applicable".
       - Manned Vehicles = "Crewed".
       - Drones/Satellites = "Uncrewed".

    Return JSON only with these exact keys:
    {
        "Market Segment": "...",
        "System Type (General)": "...",
        "System Type (Specific)": "...",
        "System Name (General)": "...",
        "System Name (Specific)": "...",
        "System Piloting": "..."
    }
    """

    class_result = call_llm(user_message, system_instruction)

    # --- B. GEOGRAPHY ---
    geo_json_str = json.dumps(GEOGRAPHY_MAPPING)

    geo_prompt = GEOGRAPHY_PROMPT.format(
        operators=VALID_OPERATORS,
        geo_mapping=geo_json_str,
        text=description
    )
    geo_result = call_llm(geo_prompt)

    # --- C. DOMESTIC CONTENT ---
    cust_c = geo_result.get("Customer Country", "Unknown")
    supp_c = geo_result.get("Supplier Country", "Unknown")

    dom_prompt = DOMESTIC_CONTENT_PROMPT.format(
        supplier_country=supp_c,
        customer_country=cust_c,
        options=DOMESTIC_CONTENT_OPTIONS,
        text=description
    )
    dom_result_raw = call_llm(dom_prompt)
    dom_val = dom_result_raw.get("Domestic Content", "Imported")

    if cust_c.lower() == supp_c.lower() and cust_c != "Unknown":
        dom_val = "Indigenous"

    if dom_val not in DOMESTIC_CONTENT_OPTIONS:
        dom_val = "Imported"

    dom_result = {"Domestic Content": dom_val}

    # --- D. FINANCIALS ---
    fin_prompt = FINANCIAL_PROMPT.format(
        program_types=PROGRAM_TYPES,
        supplier_list=", ".join(SUPPLIER_LIST),
        text=description
    )
    fin_result_raw = call_llm(fin_prompt)

    # Strict Supplier Match
    raw_llm_supplier = fin_result_raw.get("Supplier Name", "Unknown")
    matched_taxonomy_name = get_best_taxonomy_match(raw_llm_supplier)
    fin_result_raw["Supplier Name"] = matched_taxonomy_name

    derived_result = calculate_derived_fields(
        fin_result_raw, geo_result, description, contract_date_str
    )

    # --- FINAL MERGE ---
    final_output = {
        **class_result,
        **geo_result,
        **dom_result,
        **derived_result
    }

    return final_output


In [ ]:
#VALID_OPERATORS

# src/validators.py

import re
from data.taxonomy import VALID_DEPENDENCIES


def _init_validation(result: dict):
    """
    Ensures validation dict exists.
    """
    if "__validation__" not in result or not isinstance(result["__validation__"], dict):
        result["__validation__"] = {}
    return result


def _add_check(result: dict, column: str, passed: bool, reason_if_fail: str = ""):
    """
    Adds a validation check record for a single column.
    """
    result = _init_validation(result)
    result["__validation__"][column] = {
        "passed": bool(passed),
        "reason": "OK" if passed else reason_if_fail
    }
    return result


def _compute_validation_score(result: dict):
    """
    Computes validation score from __validation__ results.
    """
    v = result.get("__validation__", {})
    if not isinstance(v, dict) or len(v) == 0:
        result["Validation Score"] = 0.0
        return result

    total = len(v)
    passed = sum(1 for k in v if v[k].get("passed") is True)
    result["Validation Score"] = round((passed / total) * 100, 2)
    return result


# ==========================================================
# VALIDATORS
# ==========================================================
def validate_market_system(result: dict) -> dict:
    """
    Ensures the classification (Market Segment -> System General)
    exists in the defined Taxonomy.
    Also records validation results.
    """
    ms = result.get("Market Segment", "Unknown")
    stg = result.get("System Type (General)", "Not Applicable")
    sts = result.get("System Type (Specific)", "Not Applicable")

    # 1. Validate Market Segment exists
    if ms not in VALID_DEPENDENCIES:
        # mark fail
        result = _add_check(
            result,
            "Market Segment",
            False,
            f"Invalid Market Segment '{ms}'. Not in taxonomy."
        )

        # auto-fix output (your existing behavior)
        result["Market Segment"] = "Unknown"
        result["System Type (General)"] = "Not Applicable"
        result["System Type (Specific)"] = "Not Applicable"

        # record system checks as fail
        result = _add_check(
            result,
            "System Type (General)",
            False,
            "Market Segment invalid, so System Type (General) forced to Not Applicable."
        )
        result = _add_check(
            result,
            "System Type (Specific)",
            False,
            "Market Segment invalid, so System Type (Specific) forced to Not Applicable."
        )
        return result

    # If MS valid
    result = _add_check(result, "Market Segment", True)

    # 2. Validate System General under Market Segment
    if stg not in VALID_DEPENDENCIES[ms]:
        # fail check
        result = _add_check(
            result,
            "System Type (General)",
            False,
            f"'{stg}' is not valid under Market Segment '{ms}'."
        )
        result = _add_check(
            result,
            "System Type (Specific)",
            False,
            "System Type (General) invalid => forcing System Type (Specific) to Not Applicable."
        )

        # auto-fix (your existing behavior)
        result["System Type (General)"] = "Not Applicable"
        result["System Type (Specific)"] = "Not Applicable"
        return result

    # STG valid
    result = _add_check(result, "System Type (General)", True)

    # STS (Specific) is not strictly validated in your taxonomy dict right now
    # We'll mark it pass if it is not empty/unknown
    if sts in ["", None, "Unknown"]:
        result = _add_check(
            result,
            "System Type (Specific)",
            False,
            "System Type (Specific) missing/Unknown."
        )
    else:
        result = _add_check(result, "System Type (Specific)", True)

    return result


def validate_system_names(result: dict) -> dict:
    """
    Ensures System Name fields are consistent.
    Also records validation results.
    """
    gen = result.get("System Name (General)", "Not Applicable")
    spec = result.get("System Name (Specific)", "Not Applicable")

    # If General missing, fill with Specific
    if gen in ["Unknown", "Not Applicable", None, ""]:
        if spec not in ["Unknown", "Not Applicable", None, ""]:
            result["System Name (General)"] = spec
            result = _add_check(
                result,
                "System Name (General)",
                True,
            )
        else:
            result["System Name (General)"] = "Not Applicable"
            result = _add_check(
                result,
                "System Name (General)",
                False,
                "Both System Name (General) and (Specific) are missing/unknown."
            )
    else:
        result = _add_check(result, "System Name (General)", True)

    # If Specific missing, fill with General
    spec = result.get("System Name (Specific)", "Not Applicable")
    gen = result.get("System Name (General)", "Not Applicable")

    if spec in ["Unknown", "Not Applicable", None, ""]:
        if gen not in ["Unknown", "Not Applicable", None, ""]:
            result["System Name (Specific)"] = gen
            result = _add_check(result, "System Name (Specific)", True)
        else:
            result["System Name (Specific)"] = "Not Applicable"
            result = _add_check(
                result,
                "System Name (Specific)",
                False,
                "Both System Name (General) and (Specific) are missing/unknown."
            )
    else:
        result = _add_check(result, "System Name (Specific)", True)

    return result


def validate_program_quantity(result: dict) -> dict:
    """
    Ensures Quantity is 'Not Applicable' if Program isn't Procurement.
    Also records validation results.
    """
    prog = result.get("Program Type", "Other Service")
    qty = result.get("Quantity", "Not Applicable")

    if prog != "Procurement":
        # auto-fix
        result["Quantity"] = "Not Applicable"

        # validation check
        result = _add_check(
            result,
            "Quantity",
            True,
        )
    else:
        # For procurement, quantity should ideally be numeric or at least not N/A
        if str(qty).strip() in ["Not Applicable", "", "Unknown", "None"]:
            result = _add_check(
                result,
                "Quantity",
                False,
                "Program Type is Procurement but Quantity is missing/Not Applicable."
            )
        else:
            result = _add_check(result, "Quantity", True)

    return result


def validate_mro(result: dict) -> dict:
    """
    Ensures MRO Duration is 'Not Applicable' if Program isn't MRO.
    Also records validation results.
    """
    prog = result.get("Program Type", "Other Service")
    duration = result.get("Expected MRO Contract Duration (Months)", "Not Applicable")

    if prog != "MRO/Support":
        result["Expected MRO Contract Duration (Months)"] = "Not Applicable"
        result = _add_check(result, "Expected MRO Contract Duration (Months)", True)
    else:
        # Must not be N/A for MRO contracts
        if str(duration).strip() in ["Not Applicable", "", "Unknown", "None"]:
            result = _add_check(
                result,
                "Expected MRO Contract Duration (Months)",
                False,
                "Program Type is MRO/Support but duration is missing/Not Applicable."
            )
        else:
            result = _add_check(result, "Expected MRO Contract Duration (Months)", True)

    return result


# ==========================================================
# MASTER VALIDATION PIPELINE
# ==========================================================
def run_all_validations(result: dict, description: str) -> dict:
    """
    Master validation pipeline.
    Fixes output values + stores PASS/FAIL reasons per column.
    """
    # Ensure validation dict exists
    result = _init_validation(result)

    # 1. Validate Hierarchy
    result = validate_market_system(result)

    # 2. Validate Names
    result = validate_system_names(result)

    # 3. Validate Logical Dependencies (Quantity & MRO)
    result = validate_program_quantity(result)
    result = validate_mro(result)

    # ✅ You can add more validations here later, ex:
    # - Customer Country not Unknown
    # - Supplier Name not Unknown
    # - Value numeric
    # - Currency present, etc.

    # Score
    result = _compute_validation_score(result)

    return result


In [ ]:
## nERW APP

import streamlit as st
import pandas as pd
import numpy as np
import time
import re
import os
import sys
import shutil
import datetime
import traceback
from bs4 import BeautifulSoup
from io import BytesIO

import plotly.express as px

from selenium import webdriver
from selenium.webdriver.chrome.options import Options as ChromeOptions
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.edge.options import Options as EdgeOptions
from selenium.webdriver.edge.service import Service as EdgeService
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


st.set_page_config(
    page_title="Defense Intel Platform",
    page_icon="🛡️",
    layout="wide",
    initial_sidebar_state="expanded"
)

MEMORY_PATH = "Market Segment.xlsx"

TARGET_COLUMNS = [
    "Customer Region", "Customer Country", "Customer Operator",
    "Supplier Region", "Supplier Country", "Domestic Content",
    "Market Segment", "System Type (General)", "System Type (Specific)",
    "System Name (General)", "System Name (Specific)", "System Piloting",
    "Supplier Name", "Program Type", "Expected MRO Contract Duration (Months)",
    "Quantity", "Value Certainty", "Value (Million)", "Currency",
    "Value (USD$ Million)", "Value Note (If Any)", "G2G/B2G",
    "Signing Month", "Signing Year", "Description of Contract",
    "Additional Notes (Internal Only)", "Source Link(s)",
    "Contract Date", "Reported Date (By SGA)"
]

PROJECT_ID = st.secrets.get("PROJECT_ID", "my-test-project")
OPENAI_API_KEY = st.secrets.get("OPENAI_API_KEY", None)

if OPENAI_API_KEY:
    os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY


try:
    from src.processors import classify_record_with_memory
    from src.validators import run_all_validations
    IMPORTS_LOADED = True
    IMPORT_ERROR_MSG = None
except Exception:
    IMPORTS_LOADED = False
    IMPORT_ERROR_MSG = traceback.format_exc()


def init_state():
    if "scraped_df" not in st.session_state:
        st.session_state.scraped_df = None
    if "final_df" not in st.session_state:
        st.session_state.final_df = None
    if "logs" not in st.session_state:
        st.session_state.logs = []

init_state()


def log_event(msg: str, level: str = "INFO"):
    ts = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    st.session_state.logs.append({"time": ts, "level": level, "message": msg})


def save_memory():
    uploaded = st.session_state.mem_uploader
    if uploaded:
        try:
            with open(MEMORY_PATH, "wb") as f:
                f.write(uploaded.getbuffer())
            log_event("✅ Memory file uploaded successfully.", "SUCCESS")
            st.toast("✅ Memory loaded!", icon="💾")
        except Exception as e:
            log_event(f"❌ Memory upload failed: {e}", "ERROR")
            st.error(f"Memory upload failed: {e}")


def normalize_id(text: str):
    return str(text).replace("-", "").replace(" ", "").strip().upper()


def detect_header(paragraph_index, all_paragraphs):
    for i in range(paragraph_index, -1, -1):
        p = all_paragraphs[i]
        strong_tag = p.find("strong")
        if strong_tag:
            header_text = strong_tag.get_text(strip=True).upper()
            if header_text:
                return header_text
    return "UNKNOWN"


def get_driver():
    if sys.platform == "linux":
        options = ChromeOptions()
        options.add_argument("--headless=new")
        options.add_argument("--no-sandbox")
        options.add_argument("--disable-dev-shm-usage")
        options.add_argument("--disable-gpu")
        options.add_argument("--window-size=1920,1080")

        chromium_path = shutil.which("chromium") or "/usr/bin/chromium"
        driver_path = shutil.which("chromedriver") or "/usr/bin/chromedriver"

        options.binary_location = chromium_path
        service = ChromeService(driver_path)
        return webdriver.Chrome(service=service, options=options)

    driver_path = "driver/msedgedriver.exe"
    if os.path.exists(driver_path):
        service = EdgeService(driver_path)
    else:
        service = EdgeService()
    options = EdgeOptions()
    return webdriver.Edge(service=service, options=options)


def clean_numeric_series(series: pd.Series):
    cleaned = series.astype(str).str.replace(r"[^0-9.\-]", "", regex=True)
    return pd.to_numeric(cleaned, errors="coerce").fillna(0)


def build_export_file(df: pd.DataFrame, filename="Defense_Intel.xlsx"):
    output = BytesIO()
    with pd.ExcelWriter(output, engine="xlsxwriter") as writer:
        df.to_excel(writer, index=False, sheet_name="Defense_Contracts")
    return output.getvalue(), filename


def build_validation_table(df: pd.DataFrame):
    """
    Converts __validation__ dict into a clean table.
    """
    rows = []
    for i, r in df.iterrows():
        v = r.get("__validation__", {})
        if isinstance(v, dict):
            for col_name, meta in v.items():
                rows.append({
                    "Row": i + 1,
                    "Column": col_name,
                    "Status": "✅ PASS" if meta.get("passed") else "❌ FAIL",
                    "Reason": meta.get("reason", "")
                })
    return pd.DataFrame(rows)


# ==========================================================
# SIDEBAR
# ==========================================================
with st.sidebar:
    st.markdown("## 🛡️ Defense Intel Platform")
    st.caption("Scrape → Extract → Validate → Export")

    st.markdown("### 🔐 System")
    st.write("PROJECT_ID:", PROJECT_ID)
    st.write("OPENAI_API_KEY:", "✅ Loaded" if OPENAI_API_KEY else "❌ Missing")

    st.markdown("### 🧠 Analyst Memory")
    if os.path.exists(MEMORY_PATH):
        st.success("Memory Loaded ✅")
    else:
        st.warning("Memory Missing ❌ Upload below")

    st.file_uploader(
        "Upload Market Segment.xlsx",
        type=["xlsx"],
        key="mem_uploader",
        on_change=save_memory
    )

    st.divider()

    DEBUG_MODE = st.toggle("🔍 Debug Mode", value=False)
    MAX_URLS = st.slider("Max URLs to scrape", 1, 100, 10)
    SCRAPE_WAIT = st.slider("Wait time per URL (sec)", 1, 15, 5)

    st.divider()

    with st.expander("📜 Logs"):
        if st.session_state.logs:
            st.dataframe(pd.DataFrame(st.session_state.logs), use_container_width=True, height=300)
        else:
            st.info("No logs yet.")

    if st.button("🧹 Reset Session"):
        st.session_state.scraped_df = None
        st.session_state.final_df = None
        st.session_state.logs = []
        st.toast("✅ Reset done", icon="✅")


# ==========================================================
# MAIN
# ==========================================================
st.title("🛡️ Defense Contract Intelligence Hub")

if not IMPORTS_LOADED:
    st.error("🚨 CRITICAL ERROR: Modules failed to import.")
    st.code(IMPORT_ERROR_MSG)
    st.stop()

tab_cycle, tab_dashboard, tab_validation = st.tabs(
    ["📁 Intelligence Cycle", "📊 Dashboard & Export", "✅ Validation Center"]
)

# ==========================================================
# TAB 1: CYCLE
# ==========================================================
with tab_cycle:
    st.subheader("1️⃣ Mission Data Input")
    uploaded_file = st.file_uploader("Upload Target List (Excel)", type=["xlsx"])

    if uploaded_file:
        df_source = pd.read_excel(uploaded_file)
        st.info(f"✅ Target Loaded: {len(df_source)} rows")
        st.dataframe(df_source.head(5), use_container_width=True)

        if "Source URL" not in df_source.columns:
            st.error("❌ Input must contain 'Source URL' column.")
            st.stop()

        if "Contract Description" not in df_source.columns:
            st.error("❌ Input must contain 'Contract Description' column.")
            st.stop()

        st.subheader("2️⃣ Acquisition (Scraper)")
        if st.button("🚀 Launch Scraper", type="primary"):
            urls = df_source["Source URL"].dropna().unique().tolist()[:MAX_URLS]

            dash_pattern = r"\b[A-Z0-9]{6}\s*-\s*\d{2}\s*-\s*[A-Z0-9]\s*-\s*\d{4}\b"
            continuous_pattern = r"\b[A-Z0-9]{6}\d{2}[A-Z0-9]\d{4}\b"
            combined_pattern = f"{dash_pattern}|{continuous_pattern}"

            extracted_ids = []
            for text in df_source["Contract Description"].astype(str):
                extracted_ids.append(re.findall(combined_pattern, text.upper()))

            flat_ids = sorted(list(set([cid for sub in extracted_ids for cid in sub])))
            normalized_id_map = {normalize_id(cid): cid for cid in flat_ids}

            if not flat_ids:
                st.error("⚠️ No Contract IDs found in Contract Description")
                st.stop()

            status_box = st.status("📡 Scraping running...", expanded=True)
            progress = st.progress(0)

            driver = None
            scraped_data = []

            try:
                driver = get_driver()

                for i, url in enumerate(urls):
                    status_box.write(f"🔎 {i+1}/{len(urls)} Scanning: {url}")

                    try:
                        driver.get(url)
                        try:
                            WebDriverWait(driver, 12).until(
                                EC.presence_of_element_located((By.TAG_NAME, "p"))
                            )
                        except Exception:
                            pass

                        time.sleep(SCRAPE_WAIT)

                        html = driver.page_source
                        soup = BeautifulSoup(html, "html.parser")
                        all_paragraphs = soup.find_all("p")

                        found_count = 0

                        for p_index, p in enumerate(all_paragraphs):
                            text = p.get_text(" ", strip=True)
                            if not text:
                                continue

                            clean_text = normalize_id(text)
                            found = [orig for cln, orig in normalized_id_map.items() if cln in clean_text]

                            if found:
                                found_count += 1
                                contract_date = None
                                if "Contract Date" in df_source.columns:
                                    contract_date = df_source.loc[df_source["Source URL"] == url, "Contract Date"].iloc[0]

                                scraped_data.append({
                                    "URL": url,
                                    "Contract_Date": contract_date,
                                    "Header": detect_header(p_index, all_paragraphs),
                                    "Matched_IDs": found,
                                    "Paragraph_Text": text
                                })

                        status_box.write(f"✅ Matches found: {found_count}")

                    except Exception as e:
                        status_box.error(f"❌ Scrape failed: {e}")
                        log_event(f"Scrape failed for {url}: {e}", "ERROR")

                    progress.progress((i + 1) / len(urls))

            finally:
                if driver is not None:
                    try:
                        driver.quit()
                    except Exception:
                        pass

            if scraped_data:
                final_rows = []
                for row in scraped_data:
                    ids = row["Matched_IDs"]
                    final_rows.append({
                        "Source Link(s)": row["URL"],
                        "Contract Date": row["Contract_Date"],
                        "Header": row["Header"],
                        "Description of Contract": row["Paragraph_Text"],
                        "Supplier Name": "Multiple" if len(ids) > 1 else np.nan,
                        "Matched_ID": ", ".join(ids) if len(ids) > 1 else ids[0]
                    })

                st.session_state.scraped_df = pd.DataFrame(final_rows)
                st.success(f"✅ Scraping complete: {len(st.session_state.scraped_df)} records")
                st.dataframe(st.session_state.scraped_df.head(10), use_container_width=True)
            else:
                st.session_state.scraped_df = None
                st.warning("⚠️ No records found.")

        st.divider()
        st.subheader("3️⃣ AI Extraction + Validation")

        has_data = st.session_state.scraped_df is not None and not st.session_state.scraped_df.empty

        if not has_data:
            st.info("Waiting for scraped data...")
        else:
            if st.button("🧠 Start AI Processor", type="primary"):

                if not OPENAI_API_KEY:
                    st.error("⛔ OPENAI_API_KEY missing in Streamlit Secrets.")
                    st.stop()

                if not os.path.exists(MEMORY_PATH):
                    st.error("⛔ Memory file missing. Upload Market Segment.xlsx")
                    st.stop()

                df_in = st.session_state.scraped_df.copy()

                status = st.status("🧠 AI running...", expanded=True)
                progress_ai = st.progress(0)

                results = []

                for idx, row in df_in.iterrows():
                    desc = str(row.get("Description of Contract", ""))
                    c_date = str(row.get("Contract Date", ""))

                    status.write(f"Processing {idx+1}/{len(df_in)}")

                    try:
                        res = classify_record_with_memory(desc, c_date)
                        res = run_all_validations(res, desc)

                        row_dict = row.to_dict()
                        row_dict.pop("Supplier Name", None)
                        res.update(row_dict)

                        res["Reported Date (By SGA)"] = datetime.datetime.now().strftime("%Y-%m-%d")

                        results.append(res)

                        if DEBUG_MODE and idx == 0:
                            status.write("✅ First record output:")
                            status.json(res)

                    except Exception as e:
                        if DEBUG_MODE:
                            status.error("❌ AI error")
                            status.code(traceback.format_exc())

                        results.append({
                            "Description of Contract": desc,
                            "Additional Notes (Internal Only)": f"AI ERROR: {str(e)}",
                            "Source Link(s)": row.get("Source Link(s)", ""),
                            "Contract Date": row.get("Contract Date", ""),
                            "Reported Date (By SGA)": datetime.datetime.now().strftime("%Y-%m-%d")
                        })

                    progress_ai.progress((idx + 1) / len(df_in))

                df_out = pd.DataFrame(results)
                df_out.columns = df_out.columns.str.strip()

                # ✅ Preserve validation before column reorder
                validation_col = None
                if "__validation__" in df_out.columns:
                    validation_col = df_out["__validation__"]
                score_col = None
                if "Validation Score" in df_out.columns:
                    score_col = df_out["Validation Score"]

                # Ensure required columns exist
                for col in TARGET_COLUMNS:
                    if col not in df_out.columns:
                        df_out[col] = ""

                df_out = df_out[TARGET_COLUMNS]

                # ✅ Attach validation back
                if validation_col is not None:
                    df_out["__validation__"] = validation_col
                if score_col is not None:
                    df_out["Validation Score"] = score_col

                st.session_state.final_df = df_out
                status.update(label="✅ AI Done", state="complete", expanded=False)

                st.success("✅ Extraction + Validation Complete")
                st.dataframe(st.session_state.final_df.head(10), use_container_width=True)


# ==========================================================
# TAB 2: DASHBOARD
# ==========================================================
with tab_dashboard:
    if st.session_state.final_df is None or st.session_state.final_df.empty:
        st.info("No final results yet.")
    else:
        df = st.session_state.final_df.copy()

        st.subheader("📊 Dashboard")

        if "Value (USD$ Million)" in df.columns:
            df["Value (USD$ Million)"] = clean_numeric_series(df["Value (USD$ Million)"])

        col1, col2, col3 = st.columns(3)
        col1.metric("Records", len(df))
        col2.metric("Total Value (USD$ M)", f"{df['Value (USD$ Million)'].sum():,.2f}")
        col3.metric("Avg Validation Score", f"{df.get('Validation Score', pd.Series([0])).mean():.2f}%")

        st.divider()
        st.subheader("🧾 Data Review")
        edited = st.data_editor(df.drop(columns=["__validation__"], errors="ignore"), num_rows="dynamic",
                                use_container_width=True, height=520)

        excel_bytes, excel_name = build_export_file(edited[TARGET_COLUMNS], "Defense_Intel.xlsx")
        st.download_button(
            "⬇️ Download Excel",
            excel_bytes,
            excel_name,
            "application/vnd.openxmlformats-officedocument.spreadsheetml.sheet",
            use_container_width=True
        )


# ==========================================================
# TAB 3: VALIDATION CENTER
# ==========================================================
with tab_validation:
    st.subheader("✅ Validation Center")

    if st.session_state.final_df is None or st.session_state.final_df.empty:
        st.info("No validation data yet.")
        st.stop()

    df = st.session_state.final_df.copy()
    val_df = build_validation_table(df)

    if val_df.empty:
        st.warning("No validation report found. Ensure updated validators.py is deployed.")
        st.stop()

    # Summary
    total_checks = len(val_df)
    failed_checks = (val_df["Status"] == "❌ FAIL").sum()
    passed_checks = (val_df["Status"] == "✅ PASS").sum()

    c1, c2, c3 = st.columns(3)
    c1.metric("Total Checks", total_checks)
    c2.metric("✅ Passed", passed_checks)
    c3.metric("❌ Failed", failed_checks)

    show_failed_only = st.toggle("Show only failed validations", value=True)
    if show_failed_only:
        st.dataframe(val_df[val_df["Status"] == "❌ FAIL"], use_container_width=True, height=400)
    else:
        st.dataframe(val_df, use_container_width=True, height=400)

    st.divider()
    st.subheader("🧾 Row-wise Validation")

    max_rows = min(25, len(df))
    for idx in range(max_rows):
        row_data = df.iloc[idx]
        v = row_data.get("__validation__", {})

        failed_cols = []
        if isinstance(v, dict):
            failed_cols = [k for k, meta in v.items() if not meta.get("passed")]

        score = row_data.get("Validation Score", 0)
        title = f"Record #{idx+1} | Score: {score}% | Failed: {len(failed_cols)}"

        with st.expander(title):
            st.write("**Source Link(s):**", row_data.get("Source Link(s)", ""))
            st.write("**Contract Date:**", row_data.get("Contract Date", ""))
            st.write("**Description Snippet:**")
            st.code(str(row_data.get("Description of Contract", ""))[:600])

            if not isinstance(v, dict) or len(v) == 0:
                st.info("No validation report found for this record.")
            else:
                val_rows = []
                for col_name, meta in v.items():
                    val_rows.append({
                        "Column": col_name,
                        "Status": "✅ PASS" if meta.get("passed") else "❌ FAIL",
                        "Reason": meta.get("reason", "")
                    })
                st.dataframe(pd.DataFrame(val_rows), use_container_width=True)
